<a href="https://colab.research.google.com/github/ppastram/onerpm-publishing/blob/main/Artist_Oracle_First_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source Code

In [ ]:
#@title Required Libraries and Functions
##Import Required Packages

#Overall Utility Packages
import os
from google.colab import drive, output
from IPython.display import display, HTML
drive.mount('/content/drive')

#Data Extraction/Manipulation Packages
import pandas as pd
from pandas import json_normalize
from pandas import DataFrame as df
import numpy as np
import psycopg2
import requests
import json
from cryptography.fernet import Fernet

#API Call Packages
#!pip install requests --upgrade
!pip install spotipy --upgrade
import spotipy as sp
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import requests
!pip install ratelimit
from ratelimit import limits, sleep_and_retry #Used to limit API calls to the maximum allowed under ChartMetric's pricing plan"

#Datetime Manipulation Packages
import time
from datetime import date
from datetime import datetime
from datetime import timedelta

#Data Visualization Packages
#import matplotlib.pyplot as plt
#import seaborn as sns

#Additional Utility Functions Defined for Data Extraction, Cleaning, Transformation and Analyzing Purposes\n",
def get_day_count(start_date,end_date):
    td = end_date - start_date
    return td.days

def convertUTCDateFormat(date_str):

    utc_format = "%Y-%m-%dT00:00:00.000Z"
    reg_format = "%Y-%m-%d"
    if date_str != np.nan and date_str is not None and isinstance(date_str,str):
      try:
        datetime.strptime(date_str,utc_format).strftime(reg_format)
      except (ValueError,TypeError):
        #print("Invalid Datetime Format Entered! Please Enter UTC Format (%Y-%m-%dT00:00:00.000Z)")
        return date_str
      else:
        dt = datetime.strptime(date_str,utc_format).strftime(reg_format)
      return dt
    else:
      return np.nan

def convertFullTSFormat(date_str):

    utc_format = "%Y-%m-%dT00:00:00.000"
    alt_format = "%Y-%m-%dT00:00:00.000000"
    alt_format2 = "%Y-%m-%dT00:00:00.0000"
    alt_format3 = "%Y-%m-%dT00:00:00.00000"
    reg_format = "%Y-%m-%d"

    if date_str!= np.nan and date_str is not None and isinstance(date_str,str):
      try:
        datetime.strptime(date_str,utc_format).strftime(reg_format)
      except (ValueError):
        try:
            datetime.strptime(date_str,alt_format).strftime(reg_format)
        except (ValueError):
            try:
                datetime.strptime(date_str,alt_format2).strftime(reg_format)
            except (ValueError):
                try:
                    datetime.strptime(date_str,alt_format3).strftime(reg_format)
                except (ValueError):
                    return date_str
                else:
                    dt = datetime.strptime(date_str,alt_format3).strftime(reg_format)
            else:
                dt = datetime.strptime(date_str,alt_format2).strftime(reg_format)
        else:
            dt = datetime.strptime(date_str,alt_format).strftime(reg_format)
      except TypeError:
        return date_str
      else:
        dt = datetime.strptime(date_str,utc_format).strftime(reg_format)
      return dt
    else:
      return np.nan

def convertPartialTSFormat(date_str):

    utc_format = "%Y-%m-%dT00:00:00"
    reg_format = "%Y-%m-%d"
    if date_str != np.nan and date_str is not None and isinstance(date_str,str):
      try:
        datetime.strptime(date_str,utc_format).strftime(reg_format)
      except (ValueError,TypeError):
        #print("Invalid Datetime Format Entered! Please Enter UTC Format (%Y-%m-%dT00:00:00.000Z)")
        return date_str
      else:
        dt = datetime.strptime(date_str,utc_format).strftime(reg_format)
      return dt
    else:
      return np.nan

###Spotipy API Initialization Done Here:
SPOTIPY_CLIENT_ID = "d251fe6611524f81bcfdd0e5ae74428a" #Felix's API Client ID
SPOTIPY_CLIENT_SECRET = "9fa14ab3ef6048a1b3f995c62f4a2686" #Felix's API Client Secret

credentials = oauth2.SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET)
scope = 'user-library-read'

token = credentials.get_access_token()
sp_client = sp.Spotify(auth_manager=credentials)

###ChartMetric API Class for Tokens and Call Functions Defined Here.  See Word Document for details on what API call functions are available and what parameters you need to enter to call them.

class ChartMetricAPI():
    host = "https://api.chartmetric.com/api/token"
    key = "5qisPE8cvOfdvORPcTl5tPtOTdbEditnvs05WKHgjybsuep6wImg0ZaaA3Xu2aAA"
    access_token = None
    access_token_expiration = None
    get_token_header = {"Content-Type": "application/json"}
    token_expire_interval = 3500
    max_api_retry_count = 10
    max_calls_per_freq = 100
    rate_limit_freq = 60

    def __init__(self,host=host,key=key,token_expire_interval=token_expire_interval,max_api_retry_count=max_api_retry_count,max_calls_per_freq=max_calls_per_freq,rate_limit_freq=rate_limit_freq):
            self.host = host
            self.key=key
            self.token_expire_interval = token_expire_interval
            self.max_api_retry_count = max_api_retry_count
            self.max_calls_per_freq = max_calls_per_freq
            self.rate_limit_freq = rate_limit_freq
            try:
                self.access_token = self.getAccessToken()
                if self.access_token is None:
                    raise Exception("Request for access token failed.")
            except Exception as e:
                print(e)
            else:
                self.access_token_expiration = time.time() + self.token_expire_interval

    ##ChartMetric API Throttler Function
    @sleep_and_retry
    @limits(calls=max_calls_per_freq, period=rate_limit_freq)
    def check_limit():
      ''' Empty function just to check for calls to API '''
      return

    ##ChartMetric API Connection Function
    def getAccessToken(self):

        api_key = self.key #Insert ChartMetric API Token Here
        auth_url = self.host
        heads = {"Content-Type": "application/json"}
        refreshtokenkey = "refreshtoken"
        data = "{" + f'"{refreshtokenkey}":"{api_key}"' + "}"

        try:
            request = requests.post(auth_url,headers=heads,data=data)
            print('Access Token Request Status: ' + str(request.status_code))
            request.raise_for_status()
        except Exception as e:
            print(e)
            return None
        else:
            self.access_token_expiration = time.time() + self.token_expire_interval
            return request.json()['token']

  #Decorator defined to force token refresh when it is about to expire.
    class Decorators():
        @staticmethod
        def refreshToken(decorated):
            #Function is used to check the API token expiration and refresh if necessary
            def wrapper(api,*args,**kwargs):
                if time.time() > api.access_token_expiration:
                    api.access_token = api.getAccessToken()
                return decorated(api,*args,**kwargs)

            return wrapper

  #API Call Execution Function
    @Decorators.refreshToken
    def executeAPICall(self, api_url, headers, parameters):
        resp = requests.get(url=api_url, headers=headers, params=parameters)

        if resp.status_code == requests.codes.ok:
            return resp.json() if 'obj' in resp.json() and resp.json()['obj'] else None

        # Handling for HTTP 400 errors
        elif resp.status_code == 400:
            print(f"HTTP 400 Error for URL: {api_url} with parameters: {parameters}")
            print("Response: ", resp.text)  # Log detailed response
            return None

        # Handling for other errors
        elif resp.status_code in [502, 503, 504, 429]:
            for attempt in range(self.max_api_retry_count):
                time.sleep(1)  # Exponential backoff logic can be implemented here
                resp = requests.get(url=api_url, headers=headers, params=parameters)
                if resp.status_code == requests.codes.ok:
                    return resp.json() if 'obj' in resp.json() and resp.json()['obj'] else None
                else:
                    print(f"Retry {attempt + 1}/{self.max_api_retry_count} unsuccessful. Error Code: {resp.status_code}")
            return None

        else:
            print(f"Unhandled Error Code: {resp.status_code}")
            return None

    '''def executeAPICall(self, api_url, headers, parameters):
        if parameters == {}:
            resp = requests.get(url=api_url, headers=headers)
        else:
            resp = requests.get(url=api_url, headers=headers, params=parameters)

        if resp.status_code == requests.codes.ok:
            if 'obj' in resp.json() and resp.json()['obj'] != {}:
                return resp.json()
            else:
                print("Empty response received!")
                return None
        elif resp.status_code in [502, 503, 504]:
            time.sleep(0.5)
            retry_count = 1
            while retry_count <= self.max_api_retry_count:
                time.sleep(1)
                resp2 = requests.get(url=api_url, headers=headers, params=parameters)
                if resp2.status_code == requests.codes.ok:
                    print("API Call Retry Successful!")
                    if 'obj' in resp2.json() and resp2.json()['obj'] != {}:
                        resp_data = resp2.json()
                        break
                    else:
                        print("Empty response received!")
                        resp_data = None
                        break
                else:
                    print("API Call Retry Unsuccessful.")
                    print("Error Code: "+str(resp2.status_code))
                    resp_data = None
                    retry_count += 1
            return resp_data
        elif resp.status_code == 429:
            api_return_code = resp.status_code
            while api_return_code != 200:
                time.sleep(1)
                resp2 = requests.get(url=api_url, headers=headers, params=parameters)
                if resp2.status_code == requests.codes.ok:
                    print("API Call Retry Successful!")
                    if 'obj' in resp2.json() and resp2.json()['obj'] != {}:
                        resp_data = resp2.json()
                        api_return_code = 200
                    else:
                        print("Empty response received!")
                        resp_data = None
                        api_return_code = 200
                else:
                    print("API Call Retry Unsuccessful.")
                    print("Error Code: "+str(resp2.status_code))
            return resp_data
        else:
            print("Error Code: "+str(resp.status_code))
            print("API Call Retries Not Available For This Type of Error!")
            return None'''


  ##ChartMetric API Search Functions

    #Function for Generic Search Message From ChartMetric API
    @Decorators.refreshToken
    def searchChartMetric(self,query,limit=10,offset=0,search_type='all'):
        api_url = "https://api.chartmetric.com/api/search"
        params = {
            'q':query,
            'limit':limit,
            'offset':offset,
            'type':search_type
        }
        heads = {"Authorization":"Bearer "+self.access_token}

        return self.executeAPICall(api_url,heads,params)

    #Function for Getting ChartMetric Track ID by ISRC
    @Decorators.refreshToken
    def getCMTrackIdByISRC(self,isrc,limit=10,offset=0):
        api_url = "https://api.chartmetric.com/api/search"
        params = {
            'q':isrc,
            'limit':limit,
            'offset':offset,
            'type':'tracks'
        }
        heads = {"Authorization":"Bearer "+self.access_token}

        resp = self.executeAPICall(api_url,heads,params)

        if resp is None:
          return None
        else:
          return int(resp['obj']['tracks'][0]['cm_track'])

    #Function for Getting ChartMetric Artist ID by Artist Name
    @Decorators.refreshToken
    def getCMArtistIdByName(self,artist_name,limit=10,offset=0):
        api_url = "https://api.chartmetric.com/api/search"
        params = {
            'q':artist_name,
            'limit':limit,
            'offset':offset,
            'type':'artists'
        }
        heads = {"Authorization":"Bearer "+self.access_token}

        resp = self.executeAPICall(api_url,heads,params)

        if resp is None:
            return None
        else:
            try:
                return int(resp['obj']['artists'][0]['id'])
            except IndexError:
                print(f"No artist found with name {artist_name}")
                return None

  ##ChartMetric API Track Call Functions

    #Get Track Metadata for a given ChartMetric Track ID
    @Decorators.refreshToken
    def getTrackMetadata(self,cm_track_id):
        api_url = f"https://api.chartmetric.com/api/track/{cm_track_id}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}
        return self.executeAPICall(api_url,heads,params)

    #Generic Function to get Track Stats - Not Platform Specific
    @Decorators.refreshToken
    def getTrackStats(self,cm_track_id,platform,mode="most-history",stat_type='streams',since="2020-01-01",until=date.today().strftime("%Y-%m-%d"),interpolated='false'):
        """Available Platforms: chartmetric, spotify, youtube, shazam, tiktok, genius, soundcloud"""
        api_url = f"https://api.chartmetric.com/api/track/{cm_track_id}/{platform}/stats/{mode}"
        heads = {"Authorization":"Bearer "+self.access_token}

        #UPDATE THIS TO TAKE MORE OF THE DYNAMIC PARAMETERS
        if platform == 'spotify':
            params = {
                'since':since,
                'until':until,
                'type':stat_type,
                'interpolated':interpolated
            }
        else:
            params = {
                'since':since,
                'until':until,
                'interpolated':interpolated
            }
        return self.executeAPICall(api_url,heads,params)

    #Function to get cross-reference IDs for a track across multiple platforms based on an input of one other recognized identifier.
    @Decorators.refreshToken
    def getTrackIds(self,track_id,id_type):
        """Available ID Input Types: chartmetric, isrc, amazon, beatport, deezer, genius, itunes, shazam, spotify, soundcloud, tiktok, qq, youtube"""
        api_url = f"https://api.chartmetric.com/api/track/{id_type}/{track_id}/get-ids"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}
        return self.executeAPICall(api_url,heads,params)

    #Function to get the playlists that a track is associated with on a given date
    @Decorators.refreshToken
    def getTrackPlaylists(self,cm_track_id,platform,status,since="2022-01-01",until=date.today().strftime("%Y-%m-%d"),limit=100,offset=0,sort_column='default',editorial='true',major_curator='true'):
        api_url = f"https://api.chartmetric.com/api/track/{cm_track_id}/{platform}/{status}/playlists"
        heads = {"Authorization":"Bearer "+self.access_token}
        if status == 'current':
            params = {
                'limit':limit,
                'offset':offset,
                'sortColumn':sort_column,
                'editorial':editorial,
                'majorCurator':major_curator
                }
        else:
            params = {
                'since':since,
                'until':until,
                'offset':offset,
                'sortColumn':sort_column,
                'editorial':editorial,
                'majorCurator':major_curator
            }
        return self.executeAPICall(api_url,heads,params)

    #Get Chart Details for a single ChartMetric Track ID
    @Decorators.refreshToken
    def getTrackCharts(self,cm_track_id,chart_type,since="2022-01-01",until=date.today().strftime("%Y-%m-%d")):
        api_url = f"https://api.chartmetric.com/api/track/{cm_track_id}/{chart_type}/charts"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':since,
            'until':until
        }
        return self.executeAPICall(api_url,heads,params)

    #Get Tik Tok Video Details for a single ChartMetric Track ID
    @Decorators.refreshToken
    def getTrackTopTikTokVideos(self,cm_track_id,limit=100):
        api_url = f"https://api.chartmetric.com/api/track/{cm_track_id}/topVideos"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'limit':limit
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get the playlist snapshots for every playlist that a track is included in from ChartMetricAPI.
    @Decorators.refreshToken
    def getTrackPlaylistSnapshots(self,cm_track_id,platform,storefront='us',date=date.today().strftime("%Y-%m-%d"),limit=10,offset=0):
        api_url = f"https://api.chartmetric.com/api/track/{cm_track_id}/{platform}/playlists/snapshot"
        heads = {"Authorization":"Bearer "+self.access_token}
        if platform == 'applemusic':
            params = {
                'date':date,
                'limit':limit,
                'offset':offset,
                'storefront':storefront
            }
        else:
            params = {
                'date':date,
                'limit':limit,
                'offset':offset
            }
        return self.executeAPICall(api_url,heads,params)

  ##ChartMetric API Artist Call Functions

    #Function defined to get Chart Data from a specified platform for a single artist
    @Decorators.refreshToken
    def getArtistCharts(self,cm_artist_id,chart_type,duration,from_date='2021-12-31',to_date=date.today().strftime("%Y-%m-%d")):
        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/{chart_type}/charts"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':from_date,
            'until':to_date,
            'duration':duration
        }
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get metadata for a selected artist
    @Decorators.refreshToken
    def getArtistMetadata(self,cm_artist_id):
        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get fan metrics for a selected artist from a selected platform PLACEHOLDER
    @Decorators.refreshToken
    def getArtistFanMetrics(self,cm_artist_id,source,from_date="2021-12-21",to_date=date.today().strftime("%Y-%m-%d")):

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/stat/{source}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':from_date,
            'until':to_date,
        }
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get social audience metrics for a selected artist from a selected platform
    @Decorators.refreshToken
    def getArtistSocialAudienceStats(self,cm_artist_id,domain,audience_type,stats_type,from_date,to_date,offset=0,limit=50):

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/social-audience-stats"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'domain':domain,
            'since':from_date,
            'until':to_date,
            'audienceType':audience_type,
            'statsType':stats_type,
            'offset':offset,
            'limit':limit
        }
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get TikTok Audience Data for a single artist
    @Decorators.refreshToken
    def getArtistTikTokAudienceStats(self,cm_artist_id,as_of_date=date.today().strftime("%Y-%m-%d")):

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/tiktok-audience-stats"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'date':as_of_date,
        }
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get Instagram audience data for a selected artist
    @Decorators.refreshToken
    def getArtistInstagramAudienceStats(self,cm_artist_id,as_of_date=date.today().strftime("%Y-%m-%d"),geo_only='false'):

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/instagram-audience-stats"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'date':as_of_date,
            'geoOnly':geo_only
        }
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get the playlists that an artist is included in from ChartMetric
    @Decorators.refreshToken
    def getArtistPlaylists(self,cm_artist_id,platform,status,from_date="",to_date=date.today().strftime("%Y-%m-%d"),indie='true',editorial='true',major_curator='true',limit=100,offset=0):

        if from_date == "":
            from_dt = date.today() - pd.DateOffset(months=6)
            from_dt = from_dt.strftime("%Y-%m-%d")
        else:
            from_dt = from_date

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/{platform}/{status}/playlists"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'indie':indie,
            'since':from_dt,
            'until':to_date,
            'editorial':editorial,
            'majorCurator':major_curator,
            'offset':offset,
            'limit':limit
        }
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get the historical rankings of an artist across various platforms
    @Decorators.refreshToken
    def getHistoricalArtistRank(self,cm_artist_id,metric='cm_artist_rank',as_of_date=date.today().strftime("%Y-%m-%d")):
        '''Ranking Metrics Available: sp_popularity, sp_followers, sp_monthly_listeners, sp_listeners_to_followers_ratio, deezer_fans, cm_artist_rank, cm_artist_country_rank, cm_artist_genre_rank, cm_artist_subgenre_rank, fs_likes, fs_talks, ts_followers, ts_retweets, ins_followers, ycs_views, ycs_subscribers, ws_views, soundcloud_followers, bs_followers'''

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/past-artist-rank"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'date':as_of_date,
            'metric':metric
        }
        return self.executeAPICall(api_url,heads,params)

    #Function defined to get artist chartmetric cross-platform-performance (CPP) score and ranking
    @Decorators.refreshToken
    def getArtistCPPScore(self,cm_artist_id,stat='score',since='2020-01-01',until=date.today().strftime("%Y-%m-%d")):

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/cpp"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'stat':stat,
            'since':since,
            'until':until
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get YouTube Audience Data from ChartMetric for a given artist.
    @Decorators.refreshToken
    def getArtistYTAudienceData(self,cm_artist_id,as_of_date=date.today().strftime("%Y-%m-%d"),geo_only='false'):

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/youtube-audience-stats"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'date':as_of_date,
            'geoOnly':geo_only
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get YouTube Total Views and Market Coverage for an Artist
    @Decorators.refreshToken
    def getArtistYTViewsandMC(self,cm_artist_id):
        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/market-coverage-views/youtube"
        heads = {"Authorization":"Bearer "+self.access_token}
        params= {}
        return self.executeAPICall(api_url,heads,params)

    #Function to get Spotify Monthly Listeners by City for a given artist.
    @Decorators.refreshToken
    def getArtistSPMonthlyListenersByCity(self,cm_artist_id,since="2020-01-01",until=date.today().strftime("%Y-%m-%d"),limit=10,offset=0,show_history='true'):

        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/where-people-listen"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':since,
            'until':until,
            'limit':limit,
            'offset':offset,
            'showHistory':show_history
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get Artist IDs for given id and id type
    @Decorators.refreshToken
    def getArtistIds(self,id_type,id_value,limit=10000,offset=0,aggregate='false'):
        api_url = f"https://api.chartmetric.com/api/artist/{id_type}/{id_value}/get-ids"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'limit':limit,
            'offset':offset,
            'aggregate':aggregate
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get top tracks for an artist by platform from ChartMetricAPI
    @Decorators.refreshToken
    def getArtistTopTracksbyPlatform(self,cm_artist_id,source='tiktok',limit=100):
        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/top-tracks/{source}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'limit':limit,
            'source':source
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get tracks by artist from ChartMetricAPI.  Includes tracks where the artist is featured.
    @Decorators.refreshToken
    def getArtistTracks(self,cm_artist_id,limit=100):
        api_url = f"https://api.chartmetric.com/api/artist/{cm_artist_id}/tracks"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'limit':limit
        }
        return self.executeAPICall(api_url,heads,params)

  ##ChartMetric API Album Call Functions

    #Function to get cross-platform album IDs based on a search for a selected ID type and value using the ChartMetric API.
    @Decorators.refreshToken
    def getAlbumIds(self,id_type,id_value):
        '''Available Album ID Types: chartmetric, upc, spotify, itunes, deezer'''
        api_url = f"https://api.chartmetric.com/api/album/{id_type}/{id_value}/get-ids"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}
        return self.executeAPICall(api_url,heads,params)

    #Function to get album metadata from ChartMetricAPI
    @Decorators.refreshToken
    def getAlbumMetadata(self,cm_album_id):
        api_url = f"https://api.chartmetric.com/api/album/{cm_album_id}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}
        return self.executeAPICall(api_url,heads,params)

    #Function to get album stats from ChartMetricAPI
    @Decorators.refreshToken
    def getAlbumStats(self,cm_album_id,platform='spotify',stat='followers',since="2022-01-01",until=date.today().strftime("%Y-%m-%d"),latest='false'):
        api_url = f"https://api.chartmetric.com/api/album/{cm_album_id}/{platform}/{stat}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':since,
            'until':until,
            'latest':latest
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get playlists that an album appears on from ChartMetric API.
    @Decorators.refreshToken
    def getAlbumPlaylists(self,cm_album_id,platform,status,since="2022-01-01",until=date.today().strftime("%Y-%m-%d"),indie='false',limit=100,offset=0):
        api_url = f"https://api.chartmetric.com/api/album/{cm_album_id}/{platform}/{status}/playlists"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':since,
            'until':until,
            'indie':indie,
            'limit':limit,
            'offset':offset
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get album tracks from ChartMetricAPI
    @Decorators.refreshToken
    def getAlbumTracks(self,cm_album_id):
        api_url = f"https://api.chartmetric.com/api/album/{cm_album_id}/tracks"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}
        return self.executeAPICall(api_url,heads,params)

  ##ChartMetric API Playlist Call Functions

    #Function to get statistics around the inclusion of tracks, artists, or albums in playlists over time
    @Decorators.refreshToken
    def getPlaylistEvolutionStats(self,id_type,id_value,since="2020-01-01",until=date.today().strftime("%Y-%m-%d")):
        api_url = f"https://api.chartmetric.com/api/playlist/by/{id_type}/{id_value}/evolution"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':since,
            'until':until,
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get ChartMetric metadata around a given playlist
    @Decorators.refreshToken
    def getPlaylistMetadata(self,cm_playlist_id,platform,storefront='us'):
        api_url = f"https://api.chartmetric.com/api/playlist/{platform}/{cm_playlist_id}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}
        return self.executeAPICall(api_url,heads,params)

    #Function to get ChartMetric playlist journey or progression details
    @Decorators.refreshToken
    def getPlaylistJourneyorProgression(self,cm_playlist_id,platform,type,sortColumn='overlap',sortOrderDesc='true',limit=10,offset=0):
        api_url = f"https://api.chartmetric.com/api/playlist/{platform}/{cm_playlist_id}/journey-progression/{type}"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'sortColumn':sortColumn,
            'sortOrderDesc':sortOrderDesc,
            'limit':limit,
            'offset':offset
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get ChartMetric playlist stats for a given period of time
    @Decorators.refreshToken
    def getPlaylistStats(self,cm_playlist_id,platform,since="2022-01-01",until=date.today().strftime("%Y-%m-%d")):
        '''
            Available Platforms: spotify, deezer, youtube
            Youtube -> Playlist Views
            Deezer -> Playlist Fans
            Spotify -> Playlist Followers
        '''
        api_url = f"https://api.chartmetric.com/api/playlist/{platform}/{cm_playlist_id}/stats"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {
            'since':since,
            'until':until
        }
        return self.executeAPICall(api_url,heads,params)

    #Function to get current or historical tracks in a given playlist from ChartMetricAPI
    @Decorators.refreshToken
    def getPlaylistTracks(self,cm_playlist_id,platform,span='current',storefront='us',limit=100,offset=0,withDetails="true",since="2022-01-01",until=date.today().strftime("%Y-%m-%d")):
        api_url = f"https://api.chartmetric.com/api/playlist/{platform}/{cm_playlist_id}/{span}/tracks"
        heads = {"Authorization":"Bearer "+self.access_token}
        if span == 'past':
            if platform == 'applemusic':
                params = {
                    'limit':limit,
                    'offset':offset,
                    'withDetails':withDetails,
                    'since':since,
                    'until':until,
                    'storefront':storefront
                }
            else:
                params = {
                    'limit':limit,
                    'offset':offset,
                    'withDetails':withDetails,
                    'since':since,
                    'until':until
                }
        else:
            if platform == 'applemusic':
                params = {
                    'limit':limit,
                    'offset':offset,
                    'withDetails':withDetails,
                    'storefront':storefront
                }
            else:
                params = {
                    'limit':limit,
                    'offset':offset,
                    'withDetails':withDetails
                }
        return self.executeAPICall(api_url,heads,params)

    #Function to get snapshot details for a given playlist from ChartMetricAPI
    @Decorators.refreshToken
    def getPlaylistSnapshot(self,cm_playlist_id,platform,date=date.today().strftime("%Y-%m-%d"),storefront='us'):
        api_url = f"https://api.chartmetric.com/api/playlist/{platform}/{cm_playlist_id}/snapshot"
        heads = {"Authorization":"Bearer "+self.access_token}

        if platform == 'applemusic':
            params = {
                'storefront':storefront,
                'date':date
            }
        else:
            params = {}

        return self.executeAPICall(api_url,heads,params)

  ##ChartMetric Chart API Calls

    #Function to get snapshot details for a given playlist from ChartMetricAPI
    @Decorators.refreshToken
    def getCharts(self,chart_platform,chart_type="",chart_sub_type="",genre="",country_code='GLOBAL',city=0,cm_type_id=0,chart_duration="all_time",as_of_dt=date.today().strftime("%Y-%m-%d"),from_dt="2020-01-01",limit=100,offset=0,latest='false',countryChart='false'):
        '''
        Available Chart Platforms: airplay, amazon, applemusic, beatport, chartmetric, deezer, qq, shazam, soundcloud, spotify, tiktok, twitch, youtube, itunes
        '''

        heads = {"Authorization":"Bearer "+self.access_token}
        api_url = "https://api.chartmetric.com/api/charts"

        if chart_platform == 'airplay':
            api_endpoint = f"/airplay/{chart_type}"
            params = {
                'chart_type':chart_type,
                'date':as_of_dt,
                'since':from_dt,
                'limit':limit,
                'duration':chart_duration,
                'country_code':country_code,
                'latest':latest
            }
        elif chart_platform == 'amazon':
            if chart_sub_type != "":
                api_endpoint = f"/amazon/{chart_type}"
                params = {
                    'chart-type':chart_type,
                    'type':chart_sub_type,
                    'date':as_of_dt,
                    'code2':country_code,
                    'genre':genre,
                    'offset':offset,
                    'latest':latest
                }
            else:
                print("Invalid Chart Sub Type Selected.  Available Options Are: 'popular_track', 'new_track', 'popular_album','new_album'")
        elif chart_platform == 'applemusic':
            api_endpoint = f"/applemusic/{chart_type}"

            if city == 0 or str(city) == "":
                if genre == "":
                    params = {
                        'type':chart_sub_type,
                        'country_code':country_code,
                        'date':as_of_dt,
                        'offset':offset,
                        'latest':latest
                    }
                else:
                    params = {
                        'type':chart_sub_type,
                        'country_code':country_code,
                        'date':as_of_dt,
                        'offset':offset,
                        'latest':latest,
                        'genre':genre
                    }
            else:
                if genre == "":
                    params = {
                        'type':chart_sub_type,
                        'country_code':country_code,
                        'city_id':city,
                        'date':as_of_dt,
                        'offset':offset,
                        'latest':latest
                    }
                else:
                    params = {
                        'type':chart_sub_type,
                        'country_code':country_code,
                        'city_id':city,
                        'date':as_of_dt,
                        'offset':offset,
                        'latest':latest,
                        'genre':genre
                    }
        elif chart_platform == 'beatport':
            api_endpoint = "/beatport"
            params = {
                'date':as_of_dt,
                'genre':genre,
                'offset':offset,
                'latest':latest
            }
        elif chart_platform == 'chartmetric':
            api_endpoint = f"/{chart_sub_type}/{cm_type_id}/{chart_type}/cm-score"
            params = {
                'since':from_dt,
                'until':as_of_dt
            }
        elif chart_platform == 'deezer':
            api_endpoint = "/deezer"
            params = {
                'country_code':country_code,
                'date':as_of_dt,
                'latest':latest
            }
        elif chart_platform == 'qq':
            api_endpoint = "/qq"
            params = {
                'date':as_of_dt,
                'latest':latest
            }
        elif chart_platform == 'shazam':
            if chart_type == 'cities':
                api_endpoint = f"/shazam/{country_code}/cities"
                params = {
                    'country_code':country_code
                }
            else:
                api_endpoint = "/shazam"
                params = {
                    'country_code':country_code,
                    'date':as_of_dt,
                    'city':city,
                    'offset':offset,
                    'latest':latest
                }
        elif chart_platform == 'soundcloud':
            api_endpoint = "/soundcloud"
            params = {
                'country_code':country_code,
                'genre':genre,
                'date':as_of_dt,
                'kind':chart_type,
                'offset':offset,
                'raw':'true',
                'latest':latest
            }
        elif chart_platform == 'spotify':
            if chart_type == 'artists':
                api_endpoint = "/spotify/artists"
                params = {
                    'date':as_of_dt,
                    'type':chart_sub_type,
                    'interval':chart_duration,
                    'offset':offset,
                    'latest':latest
                }

            elif chart_type == 'fresh_find':
                api_endpoint = "/spotify/freshfind"
                params = {
                    'date':as_of_dt,
                    'latest':latest
                }

            elif chart_type == 'tracks':
                api_endpoint = "/spotify"
                params = {
                    'date':as_of_dt,
                    'country_code':country_code,
                    'type':chart_sub_type,
                    'interval':chart_duration,
                    'offset':offset,
                    'latest':latest
                }

            else:
                print('Invalid chart type selected!')
        elif chart_platform == 'tiktok':
            if chart_type == 'top_track_stats':
                api_endpoint = "/tiktok/top-tracks-stats"
                params = {
                    'date':as_of_dt,
                    'limit':limit,
                    'offset':offset
                }
            else:
                api_endpoint = f"/tiktok/{chart_type}"
                if countryChart == 'false':
                    if chart_sub_type == 'users':
                        params = {
                            'date':as_of_dt,
                            'interval':chart_duration,
                            'type':chart_sub_type,
                            'limit':limit,
                            'offset':offset,
                            'latest':latest,
                            'countryChart':countryChart
                            }
                    else:
                        params = {
                            'date':as_of_dt,
                            'interval':chart_duration,
                            'limit':limit,
                            'offset':offset,
                            'latest':latest,
                            'countryChart':countryChart
                            }
                else:
                    if chart_sub_type == 'users':
                        params = {
                            'date':as_of_dt,
                            'interval':chart_duration,
                            'type':chart_sub_type,
                            'limit':limit,
                            'offset':offset,
                            'latest':latest,
                            'countryChart':countryChart,
                            'code2':country_code
                            }
                    else:
                        params = {
                            'date':as_of_dt,
                            'interval':chart_duration,
                            'limit':limit,
                            'offset':offset,
                            'latest':latest,
                            'countryChart':countryChart,
                            'code2':country_code
                            }
        elif chart_platform == 'twitch':
            api_endpoint = "/twitch/users"
            params = {
                'date':as_of_dt,
                'since':from_dt,
                'limit':limit,
                'duration':chart_duration,
                'type':chart_type,
                'latest':latest
            }
        elif chart_platform == 'youtube':
            api_endpoint = f"/youtube/{chart_type}"
            params = {
                'country_code':country_code,
                'date':as_of_dt,
                'offset':offset,
                'latest':latest
            }
        elif chart_platform == 'itunes':
            api_endpoint = f"/itunes/{chart_type}"
            params = {
                'country_code':country_code,
                'date':as_of_dt,
                'genre':genre,
                'offset':offset,
                'latest':latest
            }
        else:
            print("Invalid Chart Platform Selected!")
            return None

        api_url = api_url+api_endpoint

        return self.executeAPICall(api_url,heads,params)

  ##ChartMetric Radio API Call Functions
    @Decorators.refreshToken
    def getAirPlayTimeSeriesStats(self,cm_id_type,cm_id,since=date.today().strftime("%Y-%m-%d")):
      api_url = f"https://api.chartmetric.com/api/radio/{cm_id_type}/{cm_id}/airplays"
      heads = {"Authorization":"Bearer "+self.access_token}
      params = {
          'since':since
      }
      return self.executeAPICall(api_url,heads,params)

    @Decorators.refreshToken
    def getBroadcastMarketPlayCounts(self,cm_id_type,cm_id,since=date.today().strftime("%Y-%m-%d")):
      api_url = f"https://api.chartmetric.com/api/radio/{cm_id_type}/{cm_id}/broadcast-markets"
      heads = {"Authorization":"Bearer "+self.access_token}
      params = {
          'since':since
      }
      return self.executeAPICall(api_url,heads,params)

    @Decorators.refreshToken
    def getRadioStationListPerCountry(self,country_code="US"):
      api_url = f"https://api.chartmetric.com/api/radio/station-list"
      heads = {"Authorization":"Bearer "+self.access_token}
      params = {
          'code2':country_code
      }
      return self.executeAPICall(api_url,heads,params)

    @Decorators.refreshToken
    def getTotalAirplays(self,cm_id_type,cm_id,cm_station_id,since=date.today().strftime("%Y-%m-%d"),limit=10):
      api_url = f"https://api.chartmetric.com/api/radio/{cm_id_type}/{cm_id}/airplay-totals"
      heads = {"Authorization":"Bearer "+self.access_token}
      params = {
          'limit':limit,
          'station':cm_station_id,
          'since':since
      }
      return self.executeAPICall(api_url,heads,params)

  ##ChartMetric City API Call Functions

    #Function to get a list of ChartMetric city ids for a given country code
    @Decorators.refreshToken
    def getCMCityIds(self,country_code):
      api_url = f"https://api.chartmetric.com/api/cities"
      heads = {"Authorization":"Bearer "+self.access_token}
      params = {
          'country_code':country_code
      }

      return self.executeAPICall(api_url,heads,params)

    #Function to get the top 100 artists for a provided ChartMetric city id
    @Decorators.refreshToken
    def getCityTopArtists(self,cm_city_id,platform):
        api_url = f"https://api.chartmetric.com/api/city/{cm_city_id}/{platform}/top-artists"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}

        return self.executeAPICall(api_url,heads,params)

    #Function to get the top 100 artists for a provided ChartMetric city id
    @Decorators.refreshToken
    def getCityTopTracks(self,cm_city_id,platform):
        api_url = f"https://api.chartmetric.com/api/city/{cm_city_id}/{platform}/top-tracks"
        heads = {"Authorization":"Bearer "+self.access_token}
        params = {}

        return self.executeAPICall(api_url,heads,params)

##Functions that use ChartMetric and Spotify API objects to pull and extract data into clean pandas dataframes are defined below

#Pull Track IDs for multiple platforms using the ISRCs from the oneRPM database data file via the ChartMetric API
def extractCMTrackIds(track_df,id_type='isrc'): #will only really work for isrcs currently

    id_reference_list = [
                         {'id_type':'chartmetric_ids','column_name':'ChartMetric Track ID'},
                         {'id_type':'spotify_ids','column_name':'Spotify ID'}
                         ]

    isrc_list = track_df[id_type].tolist()

    f_track_ids_df = pd.DataFrame()

    cm = ChartMetricAPI()

    for isrc in isrc_list:
        #print(isrc)
        if isrc != "" and isrc is not None and isrc != np.nan:
            track_json = cm.getTrackIds(isrc,id_type)
            time.sleep(0.6)
            if track_json is not None:
              if track_json['obj'] != []:
                s_track_ids_df = json_normalize(track_json['obj'])
                s_track_ids_df = s_track_ids_df[['isrc','chartmetric_ids','spotify_ids']]
                f_track_ids_df = pd.concat([f_track_ids_df,s_track_ids_df])
              else:
                s_track_ids_df = pd.DataFrame()
                s_track_ids_df['isrc'] = isrc
                s_track_ids_df['chartmetric_ids'] = np.nan
                s_track_ids_df['spotify_ids'] = np.nan

    f_track_ids_df.rename(columns={'chartmetric_ids':'ChartMetric Track ID','spotify_ids':'Spotify ID'},inplace=True)
    f_track_ids_df = f_track_ids_df.explode('ChartMetric Track ID')
    f_track_ids_df = f_track_ids_df.explode('Spotify ID')

    return f_track_ids_df

#UPDATED VERSION of extractCMTrackIDs that uses dictionary construction method via the ChartMetric API.  To be evaluated against current function for performance testing purposes
def newExtractCMTrackIds(track_df,id_type='isrc'):

    id_reference_list = [
                         {'id_type':'chartmetric_ids','column_name':'ChartMetric Track ID'},
                         {'id_type':'spotify_ids','column_name':'Spotify ID'}
                         ]

    isrc_list = track_df[id_type].tolist()
    id_list = []
    cm = ChartMetricAPI()

    for isrc in isrc_list:
        if isrc != "" and isrc is not None and isrc != np.nan:
            print(isrc)
            track_json = cm.getTrackIds(isrc,id_type)
            if track_json is not None:
              if track_json['obj'] != []:
                id_dict = {}
                id_dict['isrc'] = isrc
                id_dict['ChartMetric Track ID'] = track_json['obj'][0]['chartmetric_ids']
                id_dict['Spotify ID'] = track_json['obj'][0]['spotify_ids']
                id_list.append(id_dict)

    return pd.DataFrame(id_list)

#Pull Charts for a Track From Multiple Sources and Combine Them Into One DataFrame (WORK IN PROGRESS)
def extractCMTrackCharts(track_ids_df):

    cm = ChartMetricAPI()

    #final_df = pd.DataFrame(columns=['Platform','Chart Frequency','Chart Type','Chart Country','Chart City','Chart Genre','Chart Date','Track ISRC','ChartMetric Track ID','Track Rank'])
    final_df = pd.DataFrame()

    chart_data_ref = [
        {'chart_id':'spotify_top_daily','platform_name':'Spotify','chart_freq':'Daily','chart_category':'Top'},
        {'chart_id':'spotify_viral_daily','platform_name':'Spotify','chart_freq':'Daily','chart_category':'Viral'},
        {'chart_id':'applemusic_daily','platform_name':'Apple Music','chart_freq':'Daily','chart_category':'Top'}#,
        #{'chart_id':'amazon','platform_name':'Amazon','chart_freq':'Daily','chart_category':'Top'},
        #{'chart_id':'shazam','platform_name':'Shazam','chart_freq':'Daily','chart_category':'Top'},
        #{'chart_id':'soundcloud','platform_name':'SoundCloud','chart_freq':'Daily','chart_category':'Top'}
    ]

    cm_id_list = track_ids_df['ChartMetric Track ID'].tolist()

    track_ids_df.columns = track_ids_df.columns.str.replace(' ','_')
    track_ids_df.columns = track_ids_df.columns.str.replace('.','_')

    #for id in cm_id_list:
    for row in track_ids_df.itertuples():

        id = row.ChartMetric_Track_ID

        start_dt = row.Start_Date
        current_dt = row.End_Date
        day_cnt = get_day_count(datetime.strptime(start_dt,"%Y-%m-%d"),datetime.strptime(current_dt,"%Y-%m-%d"))

        for single_date in (datetime.strptime(start_dt,"%Y-%m-%d") + timedelta(n) for n in range(day_cnt)):
            prior_dt = single_date - timedelta(days=1)
            for ref in chart_data_ref:
                chart_json = cm.getTrackCharts(id,ref['chart_id'],since=prior_dt.strftime("%Y-%m-%d"),until=single_date.strftime("%Y-%m-%d"))
                #time.sleep(0.5)
                if chart_json is not None:
                  if chart_json != [] and chart_json != {}:
                    if chart_json['obj']['length'] != 0:
                        stage_df = json_normalize(chart_json['obj']['data'])
                        #if ref['chart_id'] in ['spotify_top_daily','spotify_viral_daily','spotify_viral_weekly','spotify_top_weekly']:
                            #stage_df.drop(['id','spotify_id','gnp','duration','chart_type','pre_rank','peak_rank','peak_date','peak_plays','total_plays','name','spotify_album_id','image_url','spotify_popularity','spotify_artist_names','track_genre','cm_artist','artist_names','code2s','artist_images','artist_covers','spotify_artist_ids','spotify_track_ids','spotify_album_ids','spotify_duration_ms','album_ids','album_names','album_upc','album_label','release_dates'],inplace=True,errors='ignore')
                            #stage_df.rename(columns={'Track Rank':'rank','Chart Country':'code2','ChartMetric Track ID':'cm_track','Track ISRC':'isrc','Chart Date':'added_at'},inplace=True,errors='ignore')
                            #stage_df['Chart City'] = np.nan
                            #stage_df['Chart Genre'] = np.nan
                        #elif ref['chart_id'] == 'applemusic_top':
                            #stage_df.drop(['id','itunes_id','gnp','pre_rank','peak_rank','peak_date','name','itunes_album_id','composer_name','image_url','track_genre','cm_artist','artist_names','code2s','artist_images','artist_covers','itunes_artist_ids','itunes_artist_names','itunes_track_ids','itunes_album_ids','storefronts','album_ids','album_names','album_upc','album_label','release_dates'],inplace=True,errors='ignore')
                            #stage_df.rename(columns={'Track Rank':'rank','Chart Country':'code2','ChartMetric Track ID':'cm_track','Track ISRC':'isrc','Chart Date':'added_at'},inplace=True,errors='ignore')
                            #stage_df['Chart City'] = np.nan
                        #elif ref['chart_id'] in ['applemusic_daily','applemusic_albums']:
                            #stage_df.drop(['id','itunes_id','gnp','pre_rank','peak_rank','peak_date','name','itunes_album_id','composer_name','image_url','track_genre','cm_artist','artist_names','code2s','artist_images','artist_covers','itunes_artist_ids','itunes_artist_names','itunes_track_ids','itunes_album_ids','storefronts','album_ids','album_names','album_upc','album_label','release_dates'],inplace=True,errors='ignore')
                            #stage_df.rename(columns={'Track Rank':'rank','Chart Country':'code2','ChartMetric Track ID':'cm_track','Track ISRC':'isrc','Chart Date':'added_at'},inplace=True,errors='ignore')
                            #stage_df['Chart City'] = np.nan
                            #stage_df['Chart Genre'] = np.nan
                        #elif ref['chart_id'] == 'amazon':
                            #stage_df.drop(['amazon','amazon_browse','pre_rank','peak_rank','peak_date','id','name','image_url','amazon_artist_id','amazon_album_id','amazon_artist_names','cm_artist','artist_names','code2s','artist_images','artist_covers','amazon_artist_ids','amazon_track_ids','amazon_genres','album_ids','album_names','album_upc','album_label'],inplace=True,errors='ignore')
                            #stage_df.rename(columns={'Track Rank':'rank','ChartMetric Track ID':'cm_track','Track ISRC':'isrc','Chart Date':'added_at','Chart Genre':'genre'},inplace=True,errors='ignore')
                            #stage_df['Chart Country'] = np.nan
                            #stage_df['Chart City'] = np.nan
                        #elif ref['chart_id'] == 'soundcloud':
                            #stage_df.drop(['id','soundcloud','gnp','pre_rank','peak_rank','peak_date','name','image_url','artist_names','album_names'])
                            #stage_df.rename(columns={'Track Rank':'rank','Chart Country':'code2','ChartMetric Track ID':'cm_track','Track ISRC':'isrc','Chart Date':'added_at'},inplace=True,errors='ignore')
                            #stage_df['Chart Country'] = np.nan
                            #stage_df['Chart City'] = np.nan
                            #stage_df['Chart Genre'] = np.nan
                        #elif ref['chart_id'] == 'shazam':
                            #stage_df.drop(['id','itunes_id','shazam_track_id','gnp','pre_rank','peak_rank','peak_date','name','image_url','itunes_track_id','cm_artist','artist_names','code2s','artist_images','artist_covers','itunes_artist_ids','itunes_artist_names','itunes_track_ids','itunes_album_ids','storefronts','album_ids','album_names','album_upc','album_label','itunes_album_id','release_dates'],inplace=True,errors='ignore')
                            #stage_df.rename(columns={'Track Rank':'rank','Chart Country':'code2','ChartMetric Track ID':'cm_track','Track ISRC':'isrc','Chart Date':'added_at'},inplace=True,errors='ignore')
                        stage_df['Platform'] = ref['platform_name']
                        stage_df['Chart Frequency'] = ref['chart_freq']
                        stage_df['Chart Type'] = ref['chart_category']
                        pd.concat([final_df,stage_df])

    return final_df

#Pull Peak Chart Ranks for a Track From Multiple Sources and Combine Them Into One DataFrame
#NOTE: Alternative to full chart details, which take too long to pull over long range for several tracks at the moment.
def extractCMTrackPeakCharts(track_ids_df):

    final_df = pd.DataFrame()

    chart_data_ref = [
        {'chart_id':'spotify_top_daily','platform_name':'Spotify','chart_freq':'Daily','chart_category':'Top','attrs':['cm_track','isrc','code2','peak_rank','peak_date']},
        {'chart_id':'spotify_viral_daily','platform_name':'Spotify','chart_freq':'Daily','chart_category':'Viral','attrs':['cm_track','isrc','code2','peak_rank','peak_date']},
        {'chart_id':'applemusic_daily','platform_name':'Apple Music','chart_freq':'Daily','chart_category':'Top','attrs':['cm_track','isrc','code2','peak_rank','peak_date']}#,
        #{'chart_id':'amazon','platform_name':'Amazon','chart_freq':'Daily','chart_category':'Top','attrs':['cm_track','isrc','code2s','peak_rank','peak_date']},
        #{'chart_id':'shazam','platform_name':'Shazam','chart_freq':'Daily','chart_category':'Top','attrs':['cm_track','isrc','code2','city','peak_rank','peak_date']},
        #{'chart_id':'soundcloud','platform_name':'SoundCloud','chart_freq':'Daily','chart_category':'Top','attrs':['cm_track','isrc','code2','peak_rank','peak_date']}
    ]

    cm_id_list = track_ids_df['ChartMetric Track ID'].tolist()
    cm = ChartMetricAPI()

    #for id in cm_id_list:
    for index, row in track_ids_df.iterrows():
        id = row["ChartMetric Track ID"]
        for ref in chart_data_ref:
            chart_json = cm.getTrackCharts(id,ref['chart_id'],since=row["Album Release Date"],until=row["End Date"]) #CHANGE SINCE PARAMETER TO PULL ONERPM ALBUM RELEASE DATE IN FUTURE
            time.sleep(1.1)
            if chart_json is not None:
                if chart_json['obj']['length'] != 0:
                    stage_df = json_normalize(chart_json['obj']['data'])
                    stage_df = stage_df[ref['attrs']]
                    if ref['chart_id'] != 'shazam':
                        stage_df['city'] = np.nan



                    stage_df['Platform'] = ref['platform_name']
                    stage_df['Chart Frequency'] = ref['chart_freq']
                    stage_df['Chart Type'] = ref['chart_category']

                    final_df = pd.concat([final_df,stage_df],join="outer")

    final_df.rename(columns={'cm_track':'ChartMetric Track ID','code2':'Chart Country','code2s':'Chart Country','city':'Chart City','peak_rank':'Track Peak Rank','peak_date':'Track Peak Date'},inplace=True,errors="ignore")
    final_df["Track Peak Date"] = final_df["Track Peak Date"].apply(convertUTCDateFormat)

    return final_df

#New Track Charts API Extraction Function to replace "extractCMTrackCharts" WORK IN PROGRESS
def extractCMConsolidatedTrackCharts(track_ids_df):
  ''''''

#Reshape the Track TikTok Video data for importation into a DataFrame (WORK IN PROGRESS: Needs to be updated to handle multiple tracks from a dataframe)
def extractCMTrackTopTikTokVideos(track_ids_df):

    cm_id_list = track_ids_df['ChartMetric Track ID'].tolist()

    df_attributes = ['video.likes','video.comments','video.followers','created_at','username','video_id','thumbnail_url','followers','cm_id','link']

    f_df = pd.DataFrame(columns=df_attributes)

    cm = ChartMetricAPI()

    for id in cm_id_list:

      tt_videos_json = cm.getTrackTopTikTokVideos()

      if tt_videos_json is not None:
        s_df = json_normalize(data=tt_videos_json['obj'],record_path="tiktokVideoHistoricalStats",meta=["created_at","username","video_id","thumbnail_url","followers","cm_id","sound_id","tiktok","tiktok_video","link"],record_prefix='video.')
        s_df['cm_track_id'] = id
        f_df = pd.concat([f_df,s_df])

    return f_df

#Function to extract ChartMetric track metadata for multiple tracks in a track dataframe.
def extractCMTrackMetadata(track_ids_df):

    cm = ChartMetricAPI()

    track_meta_list = []

    track_meta_ref = [
                      {'data_subset':'','attributes':
                          [
                            {'metadata_id':'id','column_name':'ChartMetric Track ID','in_list':False}
                          ]
                        },
                      {'data_subset':'artists','attributes':
                          [
                            {'metadata_id':'id','column_name':'CM.artist.id','in_list':True},
                            {'metadata_id':'created_at','column_name':'CM.primaryArtist.createDt','in_list':True}
                          ]
                       },
                      {'data_subset':'albums','attributes':
                          [
                            {'metadata_id':'id','column_name':'CM.album.id','in_list':True},
                            #{'metadata_id':'name','column_name':'CM.album.name','in_list':True},
                            #{'metadata_id':'upc','column_name':'CM.album.upc','in_list':True},
                            #{'metadata_id':'release_date','column_name':'CM.album.releaseDt','in_list':True},
                            #{'metadata_id':'label','column_name':'CM.album.label','in_list':True}
                          ]
                       }
                     ]

    cm_id_list = track_ids_df['ChartMetric Track ID'].tolist()

    for id in cm_id_list:
        track_json = cm.getTrackMetadata(id)
        time.sleep(0.5)
        if track_json is not None:
            track_json = track_json['obj']
            track_dict = {}

            for ref in track_meta_ref:
                for key, value in ref.items():
                    if key == 'data_subset':
                        ds = value
                    elif key == 'attributes':
                        for attr in ref[key]:
                            try:
                                if ds == '':
                                    if attr['in_list']==False:
                                        track_dict[attr['column_name']] = track_json[attr['metadata_id']]
                                    else:
                                        track_dict[attr['column_name']] = track_json[0][attr['metadata_id']]
                                else:
                                    if attr['in_list']==False:
                                        track_dict[attr['column_name']] = track_json[ds][attr['metadata_id']]
                                    else:
                                        track_dict[attr['column_name']] = track_json[ds][0][attr['metadata_id']]
                            except (KeyError, TypeError,IndexError) as e:
                                track_dict[attr['column_name']] = np.nan

            track_meta_list.append(track_dict)

    metadata_df = pd.DataFrame(track_meta_list)
    #metadata_df['CM.album.releaseDt'] = metadata_df['CM.album.releaseDt'].apply(convertUTCDateFormat)
    metadata_df['CM.primaryArtist.createDt'] = metadata_df['CM.primaryArtist.createDt'].apply(convertFullTSFormat)
    metadata_df['CM.primaryArtist.createDt'] = metadata_df['CM.primaryArtist.createDt'].apply(convertPartialTSFormat)

    return metadata_df

#Function to extract ChartMetric artist metadata for multiple tracks in a track dataframe.
def extractCMArtistMetadata(artist_ids_df):
    f_df = pd.DataFrame()

    cm_id_list = artist_ids_df['ChartMetric Artist ID'].tolist()

    cm = ChartMetricAPI()

    for id in cm_id_list:
        meta_json = cm.getArtistMetadata(id)
        time.sleep(0.5)
        if meta_json is not None:
            s_df = json_normalize(meta_json['obj'])
            f_df = pd.concat([f_df,s_df],join="outer")

    f_df.drop(columns=[
        'cm_statistics.weekly_diff.secondaryGenreRanks', #Should be pulled via separate json normalization function if required longer term
        'cm_statistics.weekly_diff_percent.secondaryGenreRanks', #Should be pulled via separate json normalization function if required longer term
        'cm_statistics.secondaryGenreRanks', #Should be pulled via separate json normalization function if required longer term
        'cm_statistics.monthly_diff.secondaryGenreRanks', #Should be pulled via separate json normalization function if required longer term
        'cm_statistics.monthly_diff_percent.secondaryGenreRanks', #Should be pulled via separate json normalization function if required longer term
        'cm_statistics.bimonthly_diff.secondaryGenreRanks', #Should be pulled via separate json normalization function if required longer term
        'cm_statistics.bimonthly_diff_percent.secondaryGenreRanks', #Should be pulled via separate json normalization function if required longer term
        'cm_statistics.monthly_diff_percent.genreRank.link_type',
        'cm_statistics.bimonthly_diff.genreRank.link_type',
        'cm_statistics.bimonthly_diff_percent.genreRank.link_type',
        'topTracks', #Should be pulled via separate json normalization function if required longer term
        'cover_url',
        'image_url',
        'type',
        'gender_title',
        'current_city_id',
        'band_members',
        'record_label',
        'genres.primary.id',
        'cm_statistics.timestamp',
        'cm_statistics.genreRank.rank',
        'cm_statistics.genreRank.percentile',
        'cm_statistics.cm_artist_rank',
        'cm_statistics.cm_artist_rank_percentile',
        'cm_statistics.fan_base_rank',
        'cm_statistics.fan_base_rank_percentile',
        'cm_statistics.engagement_rank',
        'cm_statistics.engagement_rank_percentile',
        'cm_statistics.artist_score',
        'cm_statistics.countryRank.code2',
        'cm_statistics.countryRank.rank',
        'cm_statistics.countryRank.percentile',
        'code2',#sparsely populated
        'topSongwriterId',
        'booking_agent',#sparsely populated
        'general_manager',#sparsely populated
        'press_contact',
        'rank_eg', #may be interesting to look at later?
        'rank_fb', #may be interesting to look at later?
        'topSongwriterCollaborators', #may be interesting to look at later?
        'genres.primary.link_type',
				'cm_statistics.countryRank.country',
				'cm_statistics.genreRank.id',
				'cm_statistics.genreRank.name',
				'cm_statistics.genreRank.link_type',
				'cm_statistics.sp_where_people_listen',
				'cm_statistics.sp_followers',
				'cm_statistics.deezer_fans',
				'cm_statistics.sp_popularity',
				'cm_statistics.sp_monthly_listeners',
				'cm_statistics.ins_followers',
				'cm_statistics.ycs_subscribers',
				'cm_statistics.ycs_views',
				'cm_statistics.tiktok_followers',
				'cm_statistics.tiktok_likes',
				'cm_statistics.tiktok_top_video_views',
				'cm_statistics.tiktok_track_posts',
				'cm_statistics.line_music_artist_likes',
				'cm_statistics.line_music_likes',
				'cm_statistics.line_music_plays',
				'cm_statistics.line_music_mv_plays',
				'cm_statistics.melon_artist_fans',
				'cm_statistics.melon_likes',
				'cm_statistics.melon_video_likes',
				'cm_statistics.melon_video_views',
				'cm_statistics.youtube_daily_video_views',
				'cm_statistics.youtube_monthly_video_views',
				'cm_statistics.twitch_followers',
				'cm_statistics.twitch_views',
				'cm_statistics.twitch_monthly_viewer_hours',
				'cm_statistics.twitch_weekly_viewer_hours',
				'cm_statistics.pandora_listeners_28_day',
				'cm_statistics.pandora_lifetime_stations_added',
				'cm_statistics.pandora_lifetime_streams',
				'cm_statistics.boomplay_ranking_current',
				'cm_statistics.boomplay_favorites',
				'cm_statistics.boomplay_shares',
				'cm_statistics.boomplay_comments',
				'cm_statistics.boomplay_plays',
				'cm_statistics.num_sp_editorial_playlists',
				'cm_statistics.num_sp_playlists',
				'cm_statistics.sp_playlist_total_reach',
				'cm_statistics.sp_editorial_playlist_total_reach',
				'cm_statistics.num_am_editorial_playlists',
				'cm_statistics.num_am_playlists',
				'cm_statistics.num_de_editorial_playlists',
				'cm_statistics.num_de_playlists',
				'cm_statistics.de_playlist_total_reach',
				'cm_statistics.de_editorial_playlist_total_reach',
				'cm_statistics.num_az_editorial_playlists',
				'cm_statistics.num_az_playlists',
				'cm_statistics.num_yt_editorial_playlists',
				'cm_statistics.num_yt_playlists',
				'cm_statistics.yt_playlist_total_reach',
				'cm_statistics.yt_editorial_playlist_total_reach',
				'cm_statistics.shazam_count',
				'cm_statistics.genius_pageviews',
				'cm_statistics.sp_popularity_rank',
				'cm_statistics.deezer_fans_rank',
				'cm_statistics.sp_followers_rank',
				'cm_statistics.sp_monthly_listeners_rank',
				'cm_statistics.ins_followers_rank',
				'cm_statistics.ycs_subscribers_rank',
				'cm_statistics.ycs_views_rank',
				'cm_statistics.tiktok_followers_rank',
				'cm_statistics.tiktok_track_posts_rank',
				'cm_statistics.tiktok_likes_rank',
				'cm_statistics.tiktok_top_video_views_rank',
				'cm_statistics.line_music_artist_likes_rank',
				'cm_statistics.line_music_likes_rank',
				'cm_statistics.line_music_plays_rank',
				'cm_statistics.line_music_mv_plays_rank',
				'cm_statistics.melon_artist_fans_rank',
				'cm_statistics.melon_likes_rank',
				'cm_statistics.melon_video_likes_rank',
				'cm_statistics.melon_video_views_rank',
				'cm_statistics.youtube_daily_video_views_rank',
				'cm_statistics.youtube_monthly_video_views_rank',
				'cm_statistics.twitch_followers_rank',
				'cm_statistics.twitch_views_rank',
				'cm_statistics.twitch_monthly_viewer_hours_rank',
				'cm_statistics.twitch_weekly_viewer_hours_rank',
				'cm_statistics.pandora_listeners_28_day_rank',
				'cm_statistics.pandora_lifetime_stations_added_rank',
				'cm_statistics.pandora_lifetime_streams_rank',
				'cm_statistics.boomplay_ranking_current_rank',
				'cm_statistics.boomplay_favorites_rank',
				'cm_statistics.boomplay_shares_rank',
				'cm_statistics.boomplay_comments_rank',
				'cm_statistics.boomplay_plays_rank',
				'cm_statistics.weekly_diff.countryRank.code2',
				'cm_statistics.weekly_diff.countryRank.country',
				'cm_statistics.weekly_diff.countryRank.rank',
				'cm_statistics.weekly_diff.countryRank.percentile',
				'cm_statistics.weekly_diff.genreRank.id',
				'cm_statistics.weekly_diff.genreRank.name',
				'cm_statistics.weekly_diff.genreRank.link_type',
				'cm_statistics.weekly_diff.genreRank.score',
				'cm_statistics.weekly_diff.genreRank.rank',
				'cm_statistics.weekly_diff.genreRank.percentile',
				'cm_statistics.weekly_diff.sp_where_people_listen',
				'cm_statistics.weekly_diff.sp_followers',
				'cm_statistics.weekly_diff.deezer_fans',
				'cm_statistics.weekly_diff.sp_popularity',
				'cm_statistics.weekly_diff.sp_monthly_listeners',
				'cm_statistics.weekly_diff.ins_followers',
				'cm_statistics.weekly_diff.ycs_subscribers',
				'cm_statistics.weekly_diff.ycs_views',
				'cm_statistics.weekly_diff.tiktok_followers',
				'cm_statistics.weekly_diff.tiktok_likes',
				'cm_statistics.weekly_diff.tiktok_top_video_views',
				'cm_statistics.weekly_diff.tiktok_track_posts',
				'cm_statistics.weekly_diff.line_music_artist_likes',
				'cm_statistics.weekly_diff.line_music_likes',
				'cm_statistics.weekly_diff.line_music_plays',
				'cm_statistics.weekly_diff.line_music_mv_plays',
				'cm_statistics.weekly_diff.melon_artist_fans',
				'cm_statistics.weekly_diff.melon_likes',
				'cm_statistics.weekly_diff.melon_video_likes',
				'cm_statistics.weekly_diff.melon_video_views',
				'cm_statistics.weekly_diff.youtube_daily_video_views',
				'cm_statistics.weekly_diff.youtube_monthly_video_views',
				'cm_statistics.weekly_diff.twitch_followers',
				'cm_statistics.weekly_diff.twitch_views',
				'cm_statistics.weekly_diff.twitch_monthly_viewer_hours',
				'cm_statistics.weekly_diff.twitch_weekly_viewer_hours',
				'cm_statistics.weekly_diff.pandora_listeners_28_day',
				'cm_statistics.weekly_diff.pandora_lifetime_stations_added',
				'cm_statistics.weekly_diff.pandora_lifetime_streams',
				'cm_statistics.weekly_diff.boomplay_ranking_current',
				'cm_statistics.weekly_diff.boomplay_favorites',
				'cm_statistics.weekly_diff.boomplay_shares',
				'cm_statistics.weekly_diff.boomplay_comments',
				'cm_statistics.weekly_diff.boomplay_plays',
				'cm_statistics.weekly_diff.num_sp_editorial_playlists',
				'cm_statistics.weekly_diff.num_sp_playlists',
				'cm_statistics.weekly_diff.sp_playlist_total_reach',
				'cm_statistics.weekly_diff.sp_editorial_playlist_total_reach',
				'cm_statistics.weekly_diff.num_am_editorial_playlists',
				'cm_statistics.weekly_diff.num_am_playlists',
				'cm_statistics.weekly_diff.num_de_editorial_playlists',
				'cm_statistics.weekly_diff.num_de_playlists',
				'cm_statistics.weekly_diff.de_playlist_total_reach',
				'cm_statistics.weekly_diff.de_editorial_playlist_total_reach',
				'cm_statistics.weekly_diff.num_az_editorial_playlists',
				'cm_statistics.weekly_diff.num_az_playlists',
				'cm_statistics.weekly_diff.num_yt_editorial_playlists',
				'cm_statistics.weekly_diff.num_yt_playlists',
				'cm_statistics.weekly_diff.yt_playlist_total_reach',
				'cm_statistics.weekly_diff.yt_editorial_playlist_total_reach',
				'cm_statistics.weekly_diff.shazam_count',
				'cm_statistics.weekly_diff.genius_pageviews',
				'cm_statistics.weekly_diff.cm_artist_rank',
				'cm_statistics.weekly_diff.cm_artist_rank_percentile',
				'cm_statistics.weekly_diff.fan_base_rank',
				'cm_statistics.weekly_diff.fan_base_rank_percentile',
				'cm_statistics.weekly_diff.engagement_rank',
				'cm_statistics.weekly_diff.engagement_rank_percentile',
				'cm_statistics.weekly_diff.artist_score',
				'cm_statistics.weekly_diff_percent.countryRank.code2',
				'cm_statistics.weekly_diff_percent.countryRank.country',
				'cm_statistics.weekly_diff_percent.countryRank.rank',
				'cm_statistics.weekly_diff_percent.countryRank.percentile',
				'cm_statistics.weekly_diff_percent.genreRank.id',
				'cm_statistics.weekly_diff_percent.genreRank.name',
				'cm_statistics.weekly_diff_percent.genreRank.link_type',
				'cm_statistics.weekly_diff_percent.genreRank.score',
				'cm_statistics.weekly_diff_percent.genreRank.rank',
				'cm_statistics.weekly_diff_percent.genreRank.percentile',
				'cm_statistics.weekly_diff_percent.sp_where_people_listen',
				'cm_statistics.weekly_diff_percent.sp_followers',
				'cm_statistics.weekly_diff_percent.deezer_fans',
				'cm_statistics.weekly_diff_percent.sp_popularity',
				'cm_statistics.weekly_diff_percent.sp_monthly_listeners',
				'cm_statistics.weekly_diff_percent.ins_followers',
				'cm_statistics.weekly_diff_percent.ycs_subscribers',
				'cm_statistics.weekly_diff_percent.ycs_views',
				'cm_statistics.weekly_diff_percent.tiktok_followers',
				'cm_statistics.weekly_diff_percent.tiktok_likes',
				'cm_statistics.weekly_diff_percent.tiktok_top_video_views',
				'cm_statistics.weekly_diff_percent.tiktok_track_posts',
				'cm_statistics.weekly_diff_percent.line_music_artist_likes',
				'cm_statistics.weekly_diff_percent.line_music_likes',
				'cm_statistics.weekly_diff_percent.line_music_plays',
				'cm_statistics.weekly_diff_percent.line_music_mv_plays',
				'cm_statistics.weekly_diff_percent.melon_artist_fans',
				'cm_statistics.weekly_diff_percent.melon_likes',
				'cm_statistics.weekly_diff_percent.melon_video_likes',
				'cm_statistics.weekly_diff_percent.melon_video_views',
				'cm_statistics.weekly_diff_percent.youtube_daily_video_views',
				'cm_statistics.weekly_diff_percent.youtube_monthly_video_views',
				'cm_statistics.weekly_diff_percent.twitch_followers',
				'cm_statistics.weekly_diff_percent.twitch_views',
				'cm_statistics.weekly_diff_percent.twitch_monthly_viewer_hours',
				'cm_statistics.weekly_diff_percent.twitch_weekly_viewer_hours',
				'cm_statistics.weekly_diff_percent.pandora_listeners_28_day',
				'cm_statistics.weekly_diff_percent.pandora_lifetime_stations_added',
				'cm_statistics.weekly_diff_percent.pandora_lifetime_streams',
				'cm_statistics.weekly_diff_percent.boomplay_ranking_current',
				'cm_statistics.weekly_diff_percent.boomplay_favorites',
				'cm_statistics.weekly_diff_percent.boomplay_shares',
				'cm_statistics.weekly_diff_percent.boomplay_comments',
				'cm_statistics.weekly_diff_percent.boomplay_plays',
				'cm_statistics.weekly_diff_percent.num_sp_editorial_playlists',
				'cm_statistics.weekly_diff_percent.num_sp_playlists',
				'cm_statistics.weekly_diff_percent.sp_playlist_total_reach',
				'cm_statistics.weekly_diff_percent.sp_editorial_playlist_total_reach',
				'cm_statistics.weekly_diff_percent.num_am_editorial_playlists',
				'cm_statistics.weekly_diff_percent.num_am_playlists',
				'cm_statistics.weekly_diff_percent.num_de_editorial_playlists',
				'cm_statistics.weekly_diff_percent.num_de_playlists',
				'cm_statistics.weekly_diff_percent.de_playlist_total_reach',
				'cm_statistics.weekly_diff_percent.de_editorial_playlist_total_reach',
				'cm_statistics.weekly_diff_percent.num_az_editorial_playlists',
				'cm_statistics.weekly_diff_percent.num_az_playlists',
				'cm_statistics.weekly_diff_percent.num_yt_editorial_playlists',
				'cm_statistics.weekly_diff_percent.num_yt_playlists',
				'cm_statistics.weekly_diff_percent.yt_playlist_total_reach',
				'cm_statistics.weekly_diff_percent.yt_editorial_playlist_total_reach',
				'cm_statistics.weekly_diff_percent.shazam_count',
				'cm_statistics.weekly_diff_percent.genius_pageviews',
				'cm_statistics.weekly_diff_percent.cm_artist_rank',
				'cm_statistics.weekly_diff_percent.cm_artist_rank_percentile',
				'cm_statistics.weekly_diff_percent.fan_base_rank',
				'cm_statistics.weekly_diff_percent.fan_base_rank_percentile',
				'cm_statistics.weekly_diff_percent.engagement_rank',
				'cm_statistics.weekly_diff_percent.engagement_rank_percentile',
				'cm_statistics.weekly_diff_percent.artist_score',
				'cm_statistics.monthly_diff.countryRank.code2',
				'cm_statistics.monthly_diff.countryRank.country',
				'cm_statistics.monthly_diff.countryRank.rank',
				'cm_statistics.monthly_diff.countryRank.percentile',
				'cm_statistics.monthly_diff.genreRank.id',
				'cm_statistics.monthly_diff.genreRank.name',
				'cm_statistics.monthly_diff.genreRank.link_type',
				'cm_statistics.monthly_diff.genreRank.score',
				'cm_statistics.monthly_diff.genreRank.rank',
				'cm_statistics.monthly_diff.genreRank.percentile',
				'cm_statistics.monthly_diff.sp_where_people_listen',
				'cm_statistics.monthly_diff.sp_followers',
				'cm_statistics.monthly_diff.deezer_fans',
				'cm_statistics.monthly_diff.sp_popularity',
				'cm_statistics.monthly_diff.sp_monthly_listeners',
				'cm_statistics.monthly_diff.ins_followers',
				'cm_statistics.monthly_diff.ycs_subscribers',
				'cm_statistics.monthly_diff.ycs_views',
				'cm_statistics.monthly_diff.tiktok_followers',
				'cm_statistics.monthly_diff.tiktok_likes',
				'cm_statistics.monthly_diff.tiktok_top_video_views',
				'cm_statistics.monthly_diff.tiktok_track_posts',
				'cm_statistics.monthly_diff.line_music_artist_likes',
				'cm_statistics.monthly_diff.line_music_likes',
				'cm_statistics.monthly_diff.line_music_plays',
				'cm_statistics.monthly_diff.line_music_mv_plays',
				'cm_statistics.monthly_diff.melon_artist_fans',
				'cm_statistics.monthly_diff.melon_likes',
				'cm_statistics.monthly_diff.melon_video_likes',
				'cm_statistics.monthly_diff.melon_video_views',
				'cm_statistics.monthly_diff.youtube_daily_video_views',
				'cm_statistics.monthly_diff.youtube_monthly_video_views',
				'cm_statistics.monthly_diff.twitch_followers',
				'cm_statistics.monthly_diff.twitch_views',
				'cm_statistics.monthly_diff.twitch_monthly_viewer_hours',
				'cm_statistics.monthly_diff.twitch_weekly_viewer_hours',
				'cm_statistics.monthly_diff.pandora_listeners_28_day',
				'cm_statistics.monthly_diff.pandora_lifetime_stations_added',
				'cm_statistics.monthly_diff.pandora_lifetime_streams',
				'cm_statistics.monthly_diff.boomplay_ranking_current',
				'cm_statistics.monthly_diff.boomplay_favorites',
				'cm_statistics.monthly_diff.boomplay_shares',
				'cm_statistics.monthly_diff.boomplay_comments',
				'cm_statistics.monthly_diff.boomplay_plays',
				'cm_statistics.monthly_diff.num_sp_editorial_playlists',
				'cm_statistics.monthly_diff.num_sp_playlists',
				'cm_statistics.monthly_diff.sp_playlist_total_reach',
				'cm_statistics.monthly_diff.sp_editorial_playlist_total_reach',
				'cm_statistics.monthly_diff.num_am_editorial_playlists',
				'cm_statistics.monthly_diff.num_am_playlists',
				'cm_statistics.monthly_diff.num_de_editorial_playlists',
				'cm_statistics.monthly_diff.num_de_playlists',
				'cm_statistics.monthly_diff.de_playlist_total_reach',
				'cm_statistics.monthly_diff.de_editorial_playlist_total_reach',
				'cm_statistics.monthly_diff.num_az_editorial_playlists',
				'cm_statistics.monthly_diff.num_az_playlists',
				'cm_statistics.monthly_diff.num_yt_editorial_playlists',
				'cm_statistics.monthly_diff.num_yt_playlists',
				'cm_statistics.monthly_diff.yt_playlist_total_reach',
				'cm_statistics.monthly_diff.yt_editorial_playlist_total_reach',
				'cm_statistics.monthly_diff.shazam_count',
				'cm_statistics.monthly_diff.genius_pageviews',
				'cm_statistics.monthly_diff.cm_artist_rank',
				'cm_statistics.monthly_diff.cm_artist_rank_percentile',
				'cm_statistics.monthly_diff.fan_base_rank',
				'cm_statistics.monthly_diff.fan_base_rank_percentile',
				'cm_statistics.monthly_diff.engagement_rank',
				'cm_statistics.monthly_diff.engagement_rank_percentile',
				'cm_statistics.monthly_diff.artist_score',
				'cm_statistics.monthly_diff_percent.countryRank.code2',
				'cm_statistics.monthly_diff_percent.countryRank.country',
				'cm_statistics.monthly_diff_percent.countryRank.rank',
				'cm_statistics.monthly_diff_percent.countryRank.percentile',
				'cm_statistics.monthly_diff_percent.genreRank.id',
				'cm_statistics.monthly_diff_percent.genreRank.name',
				'cm_statistics.monthly_diff_percent.genreRank.score',
				'cm_statistics.monthly_diff_percent.genreRank.rank',
				'cm_statistics.monthly_diff_percent.genreRank.percentile',
				'cm_statistics.monthly_diff_percent.sp_where_people_listen',
				'cm_statistics.monthly_diff_percent.sp_followers',
				'cm_statistics.monthly_diff_percent.deezer_fans',
				'cm_statistics.monthly_diff_percent.sp_popularity',
				'cm_statistics.monthly_diff_percent.sp_monthly_listeners',
				'cm_statistics.monthly_diff_percent.ins_followers',
				'cm_statistics.monthly_diff_percent.ycs_subscribers',
				'cm_statistics.monthly_diff_percent.ycs_views',
				'cm_statistics.monthly_diff_percent.tiktok_followers',
				'cm_statistics.monthly_diff_percent.tiktok_likes',
				'cm_statistics.monthly_diff_percent.tiktok_top_video_views',
				'cm_statistics.monthly_diff_percent.tiktok_track_posts',
				'cm_statistics.monthly_diff_percent.line_music_artist_likes',
				'cm_statistics.monthly_diff_percent.line_music_likes',
				'cm_statistics.monthly_diff_percent.line_music_plays',
				'cm_statistics.monthly_diff_percent.line_music_mv_plays',
				'cm_statistics.monthly_diff_percent.melon_artist_fans',
				'cm_statistics.monthly_diff_percent.melon_likes',
				'cm_statistics.monthly_diff_percent.melon_video_likes',
				'cm_statistics.monthly_diff_percent.melon_video_views',
				'cm_statistics.monthly_diff_percent.youtube_daily_video_views',
				'cm_statistics.monthly_diff_percent.youtube_monthly_video_views',
				'cm_statistics.monthly_diff_percent.twitch_followers',
				'cm_statistics.monthly_diff_percent.twitch_views',
				'cm_statistics.monthly_diff_percent.twitch_monthly_viewer_hours',
				'cm_statistics.monthly_diff_percent.twitch_weekly_viewer_hours',
				'cm_statistics.monthly_diff_percent.pandora_listeners_28_day',
				'cm_statistics.monthly_diff_percent.pandora_lifetime_stations_added',
				'cm_statistics.monthly_diff_percent.pandora_lifetime_streams',
				'cm_statistics.monthly_diff_percent.boomplay_ranking_current',
				'cm_statistics.monthly_diff_percent.boomplay_favorites',
				'cm_statistics.monthly_diff_percent.boomplay_shares',
				'cm_statistics.monthly_diff_percent.boomplay_comments',
				'cm_statistics.monthly_diff_percent.boomplay_plays',
				'cm_statistics.monthly_diff_percent.num_sp_editorial_playlists',
				'cm_statistics.monthly_diff_percent.num_sp_playlists',
				'cm_statistics.monthly_diff_percent.sp_playlist_total_reach',
				'cm_statistics.monthly_diff_percent.sp_editorial_playlist_total_reach',
				'cm_statistics.monthly_diff_percent.num_am_editorial_playlists',
				'cm_statistics.monthly_diff_percent.num_am_playlists',
				'cm_statistics.monthly_diff_percent.num_de_editorial_playlists',
				'cm_statistics.monthly_diff_percent.num_de_playlists',
				'cm_statistics.monthly_diff_percent.de_playlist_total_reach',
				'cm_statistics.monthly_diff_percent.de_editorial_playlist_total_reach',
				'cm_statistics.monthly_diff_percent.num_az_editorial_playlists',
				'cm_statistics.monthly_diff_percent.num_az_playlists',
				'cm_statistics.monthly_diff_percent.num_yt_editorial_playlists',
				'cm_statistics.monthly_diff_percent.num_yt_playlists',
				'cm_statistics.monthly_diff_percent.yt_playlist_total_reach',
				'cm_statistics.monthly_diff_percent.yt_editorial_playlist_total_reach',
				'cm_statistics.monthly_diff_percent.shazam_count',
				'cm_statistics.monthly_diff_percent.genius_pageviews',
				'cm_statistics.monthly_diff_percent.cm_artist_rank',
				'cm_statistics.monthly_diff_percent.cm_artist_rank_percentile',
				'cm_statistics.monthly_diff_percent.fan_base_rank',
				'cm_statistics.monthly_diff_percent.fan_base_rank_percentile',
				'cm_statistics.monthly_diff_percent.engagement_rank',
				'cm_statistics.monthly_diff_percent.engagement_rank_percentile',
				'cm_statistics.monthly_diff_percent.artist_score',
				'cm_statistics.bimonthly_diff.countryRank.code2',
				'cm_statistics.bimonthly_diff.countryRank.country',
				'cm_statistics.bimonthly_diff.countryRank.rank',
				'cm_statistics.bimonthly_diff.countryRank.percentile',
				'cm_statistics.bimonthly_diff.genreRank.id',
				'cm_statistics.bimonthly_diff.genreRank.name',
				'cm_statistics.bimonthly_diff.genreRank.score',
				'cm_statistics.bimonthly_diff.genreRank.rank',
				'cm_statistics.bimonthly_diff.genreRank.percentile',
				'cm_statistics.bimonthly_diff.sp_where_people_listen',
				'cm_statistics.bimonthly_diff.sp_followers',
				'cm_statistics.bimonthly_diff.deezer_fans',
				'cm_statistics.bimonthly_diff.sp_popularity',
				'cm_statistics.bimonthly_diff.sp_monthly_listeners',
				'cm_statistics.bimonthly_diff.ins_followers',
				'cm_statistics.bimonthly_diff.ycs_subscribers',
				'cm_statistics.bimonthly_diff.ycs_views',
				'cm_statistics.bimonthly_diff.tiktok_followers',
				'cm_statistics.bimonthly_diff.tiktok_likes',
				'cm_statistics.bimonthly_diff.tiktok_top_video_views',
				'cm_statistics.bimonthly_diff.tiktok_track_posts',
				'cm_statistics.bimonthly_diff.line_music_artist_likes',
				'cm_statistics.bimonthly_diff.line_music_likes',
				'cm_statistics.bimonthly_diff.line_music_plays',
				'cm_statistics.bimonthly_diff.line_music_mv_plays',
				'cm_statistics.bimonthly_diff.melon_artist_fans',
				'cm_statistics.bimonthly_diff.melon_likes',
				'cm_statistics.bimonthly_diff.melon_video_likes',
				'cm_statistics.bimonthly_diff.melon_video_views',
				'cm_statistics.bimonthly_diff.youtube_daily_video_views',
				'cm_statistics.bimonthly_diff.youtube_monthly_video_views',
				'cm_statistics.bimonthly_diff.twitch_followers',
				'cm_statistics.bimonthly_diff.twitch_views',
				'cm_statistics.bimonthly_diff.twitch_monthly_viewer_hours',
				'cm_statistics.bimonthly_diff.twitch_weekly_viewer_hours',
				'cm_statistics.bimonthly_diff.pandora_listeners_28_day',
				'cm_statistics.bimonthly_diff.pandora_lifetime_stations_added',
				'cm_statistics.bimonthly_diff.pandora_lifetime_streams',
				'cm_statistics.bimonthly_diff.boomplay_ranking_current',
				'cm_statistics.bimonthly_diff.boomplay_favorites',
				'cm_statistics.bimonthly_diff.boomplay_shares',
				'cm_statistics.bimonthly_diff.boomplay_comments',
				'cm_statistics.bimonthly_diff.boomplay_plays',
				'cm_statistics.bimonthly_diff.num_sp_editorial_playlists',
				'cm_statistics.bimonthly_diff.num_sp_playlists',
				'cm_statistics.bimonthly_diff.sp_playlist_total_reach',
				'cm_statistics.bimonthly_diff.sp_editorial_playlist_total_reach',
				'cm_statistics.bimonthly_diff.num_am_editorial_playlists',
				'cm_statistics.bimonthly_diff.num_am_playlists',
				'cm_statistics.bimonthly_diff.num_de_editorial_playlists',
				'cm_statistics.bimonthly_diff.num_de_playlists',
				'cm_statistics.bimonthly_diff.de_playlist_total_reach',
				'cm_statistics.bimonthly_diff.de_editorial_playlist_total_reach',
				'cm_statistics.bimonthly_diff.num_az_editorial_playlists',
				'cm_statistics.bimonthly_diff.num_az_playlists',
				'cm_statistics.bimonthly_diff.num_yt_editorial_playlists',
				'cm_statistics.bimonthly_diff.num_yt_playlists',
				'cm_statistics.bimonthly_diff.yt_playlist_total_reach',
				'cm_statistics.bimonthly_diff.yt_editorial_playlist_total_reach',
				'cm_statistics.bimonthly_diff.shazam_count',
				'cm_statistics.bimonthly_diff.genius_pageviews',
				'cm_statistics.bimonthly_diff.cm_artist_rank',
				'cm_statistics.bimonthly_diff.cm_artist_rank_percentile',
				'cm_statistics.bimonthly_diff.fan_base_rank',
				'cm_statistics.bimonthly_diff.fan_base_rank_percentile',
				'cm_statistics.bimonthly_diff.engagement_rank',
				'cm_statistics.bimonthly_diff.engagement_rank_percentile',
				'cm_statistics.bimonthly_diff.artist_score',
				'cm_statistics.bimonthly_diff_percent.countryRank.code2',
				'cm_statistics.bimonthly_diff_percent.countryRank.country',
				'cm_statistics.bimonthly_diff_percent.countryRank.rank',
				'cm_statistics.bimonthly_diff_percent.countryRank.percentile',
				'cm_statistics.bimonthly_diff_percent.genreRank.id',
				'cm_statistics.bimonthly_diff_percent.genreRank.name',
				'cm_statistics.bimonthly_diff_percent.genreRank.score',
				'cm_statistics.bimonthly_diff_percent.genreRank.rank',
				'cm_statistics.bimonthly_diff_percent.genreRank.percentile',
				'cm_statistics.bimonthly_diff_percent.sp_where_people_listen',
				'cm_statistics.bimonthly_diff_percent.sp_followers',
				'cm_statistics.bimonthly_diff_percent.deezer_fans',
				'cm_statistics.bimonthly_diff_percent.sp_popularity',
				'cm_statistics.bimonthly_diff_percent.sp_monthly_listeners',
				'cm_statistics.bimonthly_diff_percent.ins_followers',
				'cm_statistics.bimonthly_diff_percent.ycs_subscribers',
				'cm_statistics.bimonthly_diff_percent.ycs_views',
				'cm_statistics.bimonthly_diff_percent.tiktok_followers',
				'cm_statistics.bimonthly_diff_percent.tiktok_likes',
				'cm_statistics.bimonthly_diff_percent.tiktok_top_video_views',
				'cm_statistics.bimonthly_diff_percent.tiktok_track_posts',
				'cm_statistics.bimonthly_diff_percent.line_music_artist_likes',
				'cm_statistics.bimonthly_diff_percent.line_music_likes',
				'cm_statistics.bimonthly_diff_percent.line_music_plays',
				'cm_statistics.bimonthly_diff_percent.line_music_mv_plays',
				'cm_statistics.bimonthly_diff_percent.melon_artist_fans',
				'cm_statistics.bimonthly_diff_percent.melon_likes',
				'cm_statistics.bimonthly_diff_percent.melon_video_likes',
				'cm_statistics.bimonthly_diff_percent.melon_video_views',
				'cm_statistics.bimonthly_diff_percent.youtube_daily_video_views',
				'cm_statistics.bimonthly_diff_percent.youtube_monthly_video_views',
				'cm_statistics.bimonthly_diff_percent.twitch_followers',
				'cm_statistics.bimonthly_diff_percent.twitch_views',
				'cm_statistics.bimonthly_diff_percent.twitch_monthly_viewer_hours',
				'cm_statistics.bimonthly_diff_percent.twitch_weekly_viewer_hours',
				'cm_statistics.bimonthly_diff_percent.pandora_listeners_28_day',
				'cm_statistics.bimonthly_diff_percent.pandora_lifetime_stations_added',
				'cm_statistics.bimonthly_diff_percent.pandora_lifetime_streams',
				'cm_statistics.bimonthly_diff_percent.boomplay_ranking_current',
				'cm_statistics.bimonthly_diff_percent.boomplay_favorites',
				'cm_statistics.bimonthly_diff_percent.boomplay_shares',
				'cm_statistics.bimonthly_diff_percent.boomplay_comments','cm_statistics.bimonthly_diff_percent.boomplay_plays','cm_statistics.bimonthly_diff_percent.num_sp_editorial_playlists',
				'cm_statistics.bimonthly_diff_percent.num_sp_playlists','cm_statistics.bimonthly_diff_percent.sp_playlist_total_reach','cm_statistics.bimonthly_diff_percent.sp_editorial_playlist_total_reach',
				'cm_statistics.bimonthly_diff_percent.num_am_editorial_playlists','cm_statistics.bimonthly_diff_percent.num_am_playlists','cm_statistics.bimonthly_diff_percent.num_de_editorial_playlists',
				'cm_statistics.bimonthly_diff_percent.num_de_playlists','cm_statistics.bimonthly_diff_percent.de_playlist_total_reach','cm_statistics.bimonthly_diff_percent.de_editorial_playlist_total_reach',
				'cm_statistics.bimonthly_diff_percent.num_az_editorial_playlists','cm_statistics.bimonthly_diff_percent.num_az_playlists','cm_statistics.bimonthly_diff_percent.num_yt_editorial_playlists',
				'cm_statistics.bimonthly_diff_percent.num_yt_playlists','cm_statistics.bimonthly_diff_percent.yt_playlist_total_reach','cm_statistics.bimonthly_diff_percent.yt_editorial_playlist_total_reach',
				'cm_statistics.bimonthly_diff_percent.shazam_count','cm_statistics.bimonthly_diff_percent.genius_pageviews','cm_statistics.bimonthly_diff_percent.cm_artist_rank',
				'cm_statistics.bimonthly_diff_percent.cm_artist_rank_percentile','cm_statistics.bimonthly_diff_percent.fan_base_rank','cm_statistics.bimonthly_diff_percent.fan_base_rank_percentile',
				'cm_statistics.bimonthly_diff_percent.engagement_rank','cm_statistics.bimonthly_diff_percent.engagement_rank_percentile','cm_statistics.bimonthly_diff_percent.artist_score',
        'cm_statistics.genreRanks',
        'created_at',
        'artist_stage.career_stage',
        'artist_stage.carrer_stage_score',
        'artist_stage.career_growth',
        'artist_stage.career_growth_score',
        'band',
        'gender',
        'isni',
        'tags',
        'description',
        'name'
        ]
        ,inplace=True,errors="ignore")

    f_df.rename(columns={'id':'ChartMetric Artist ID',
                         'name':'ChartMetric Artist Name',
                         'cm_artist_rank':'ChartMetric Current Artist Rank',
                         'cm_artist_score':'ChartMetric Artist Score',
                         'description':'ChartMetric Artist Description',
                         'genres.primary.name':'cm.artist.primaryGenre',
                         'genres.primary.score':'ChartMetric Artist Primary Genre Score',
                         'genres.secondary':'ChartMetric Artist Secondary Genre',
                         'genres.sub':'ChartMetric Artist Sub Genre'}
                ,inplace=True,errors="ignore")

    #f_df['CM.album.releaseDt'] = f_df['ChartMetric Artist Creation Date'].apply(convertUTCDateFormat)
    #f_df['CM.primaryArtist.createDt'] = f_df['ChartMetric Artist Creation Date'].apply(convertFullTSFormat)

    return f_df

#Function to extract key statistics on all songs within a given dataset
def extractCMTrackStats(track_ids_df):

    stat_platforms = [
                    {'platform':'spotify','stat':'streams'},
                    {'platform':'spotify','stat':'popularity'},
                    {'platform':'youtube','stat':'views'},
                    {'platform':'shazam','stat':'play_counts'},
                    {'platform':'soundcloud','stat':'play_counts'},
                    {'platform':'tiktok','stat':'post_counts'}
                    ]

    stat_attributes = [
        'Track Metric Platform',
        'Track Metric Name',
        'ChartMetric Track ID',
        'value',
        'timestp',
        'diff'
    ]

    track_ids_df.columns = track_ids_df.columns.str.replace(' ','_')
    track_ids_df.columns = track_ids_df.columns.str.replace('.','_')

    f_df = pd.DataFrame(columns=stat_attributes)

    cm = ChartMetricAPI()

    for row in track_ids_df.itertuples():
        print(row)
        print(str(row.Index) + ' ' + str(row.ChartMetric_Track_ID))
        id = row.ChartMetric_Track_ID
        if id != "" and id != np.nan and id is not None:
            extract_dt = row.Start_Date
            current_dt = row.End_Date
            for p in stat_platforms:
                track_json = cm.getTrackStats(id,platform=p['platform'],stat_type=p['stat'],since=extract_dt,until=current_dt)
                #time.sleep(0.5) #FIND MORE EFFICIENT WAY TO THROTTLE API
                if track_json is not None:
                  if track_json['obj'] != []:
                    s_df = json_normalize(track_json['obj'],record_path='data')
                    s_df['Track Metric Platform'] = p['platform']
                    s_df['Track Metric Name'] = p['stat']
                    s_df['ChartMetric Track ID'] = id

                    f_df = pd.concat([f_df,s_df],join="inner")

    f_df['timestp'] = f_df['timestp'].apply(convertUTCDateFormat)
    f_df.rename(columns={'timestp':'Track Metric Date','value':'Track Metric Value','diff':'Track Metric Difference'},inplace=True)

    return f_df

#Function to extract details for all playlists that a track appears on within a certain timeframe: WORK IN PROGRESS
def extractCMTrackPlaylists(track_df):
    ''''''
    f_df = pd.DataFrame()
    playlist_ref = [
        {'platform':'spotify','attrs':[
            'playlist.playlist_id',
            'playlist.peak_position',
            'playlist.position',
            'playlist.added_at',
            'playlist.removed_at',
            'playlist.period',
            'playlist.followers',
            'playlist.id',
            'playlist.name',
            'playlist.description',
            'playlist.sys_last_updated',
            'playlist.personalized',
            'playlist.owner_name',
            'playlist.owner_id',
            'playlist.official',
            'playlist.tags',
            'playlist.num_track',
            'playlist.catalog',
            'playlist.active_ratio'
        ]},
        {'platform':'applemusic','attrs':[
        ['playlist','playlist_id'],
        ['playlist','track_id'],
        ['playlist','min_position'],
        ['playlist','min_added_at'],
        ['playlist','min_removed_at'],
        ['playlist','num_countries'],
        ['playlist','peak_position'],
        ['playlist','max_period'],
        ['playlist','id'],
        ['playlist','name'],
        ['playlist','description'],
        ['playlist','curator_id'],
        ['playlist','with_storefronts'],
        ['playlist','sys_last_updated'],
        ['playlist','code2'],
        ['playlist','owner_id'],
        ['playlist','owner_name'],
        ['playlist','official'],
        ['playlist','storefronts'],
        ['playlist','last_added'],
        ['playlist','last_removed'],
        ['playlist','num_track'],
        ['playlist','tags'],
        ['playlist','catalog'],
        ['playlist','active_ratio'],
        ['playlist','period'],
        ['playlist','position'],
        ['playlist','added_at'],
        ['playlist','removed_at']]
         },
        {'platform':'deezer','attrs':[
            'playlist.playlist_id',
            'playlist.peak_position',
            'playlist.position',
            'playlist.added_at',
            'playlist.removed_at',
            'playlist.period',
            'playlist.followers',
            'playlist.id',
            'playlist.name',
            'playlist.description',
            'playlist.sys_last_updated',
            'playlist.owner_name',
            'playlist.owner_id',
            'playlist.official',
            'playlist.tags',
            'playlist.num_track',
            'playlist.catalog',
            'playlist.active_ratio']}
    ]

    cm = ChartMetricAPI()

    cm_track_df = track_df[['ChartMetric Track ID','release_date']].drop_duplicates().dropna(how="all")

    track_df.columns = track_df.columns.str.replace(' ','_')
    track_df.columns = track_df.columns.str.replace('.','_')

    for index, row in cm_track_df.itertuples():
        from_dt = row.Album_Release_Date
        if row.ChartMetric_Track_ID != np.nan:
            track_id = row.ChartMetric_Track_ID
            for ref in playlist_ref:
                pl_json = cm.getTrackPlaylists(track_id,platform=ref['platform'],status='past',since=from_dt)
                time.sleep(0.7)
                if pl_json is not None and pl_json['obj']!=[]:
                    if ref['platform'] == 'applemusic':
                        s_df = json_normalize(pl_json['obj'],record_path=["playlist","details"],meta=ref['attrs'])
                    else:
                        s_df = json_normalize(pl_json['obj'])
                        s_df = s_df[ref['attrs']]
                    s_df['playlist.platform'] = ref['platform']
                    s_df['ChartMetric Track ID'] = row.ChartMetric_Track_ID
                    f_df = pd.concat([f_df,s_df])

    return f_df

#Function to extract cross-platform artist IDs for the artist names in the onerpm dataset
def extractCMArtistIDs(artist_df):
    cm = ChartMetricAPI()
    id_reference_list = [
                            {'id':'chartmetric_id','column_name':'ChartMetric Artist ID','in_list':False},
                            {'id':'artist_name','column_name':'Artist Name','in_list':False},
                            {'id':'spotify_artist_ids','column_name':'Spotify ID','in_list':True},
                            {'id':'itunes_artist_ids','column_name':'Apple ID','in_list':True},
                            {'id':'deezer_artist_ids','column_name':'Deezer ID','in_list':True},
                            {'id':'amazon_artist_ids','column_name':'Amazon ID','in_list':True}
                        ]

    artist_id_list = []

    for index, row in artist_df.iterrows():
        print(str(index) + ' ' + str(row['name']))
        cm_artist_id = cm.getCMArtistIdByName(row['name'],limit=100,offset=0)
        time.sleep(1)
        if cm_artist_id is not None:
            artist_id_json = cm.getArtistIds('chartmetric',cm_artist_id,limit=10000,offset=0,aggregate='true')
            time.sleep(1)
            artist_dict = {}
            if artist_id_json is not None:
                for ref in id_reference_list:
                    if ref['in_list'] == True:
                        try:
                            if len(artist_id_json['obj'][0][ref['id']])==1 and ref['in_list'] == True:
                                artist_dict[ref['column_name']] = artist_id_json['obj'][0][ref['id']][0]
                            else:
                                artist_dict[ref['column_name']] = artist_id_json['obj'][0][ref['id']]
                        except TypeError:
                            artist_dict[ref['column_name']] = artist_id_json['obj'][0][ref['id']]
                    else:
                        artist_dict[ref['column_name']] = artist_id_json['obj'][0][ref['id']]
                artist_id_list.append(artist_dict)
            else:
                print("No IDs Found")
    return pd.DataFrame(artist_id_list)

#Function to extract fan metrics for a collection of ChartMetric artist IDs that are pulled via the extractCMArtistIDs function from the ChartMetric API.
def extractCMArtistFanMetrics(artists_df,from_dt="",to_dt=""):

    fan_metric_ref = [
                            {'platform_id':'spotify','metrics':['followers','popularity','listeners','followers_to_listeners_ratio']},
                            {'platform_id':'facebook','metrics':['likes','talks']},
                            {'platform_id':'twitter','metrics':['followers','retweets']},
                            {'platform_id':'youtube_channel','metrics':[
                                                                        'subscribers',
                                                                        'views',
                                                                        'comments',
                                                                        'videos']},
                            {'platform_id':'youtube_artist','metrics':[
                                                                       'daily_views',
                                                                       'monthly_views'
                                                                       ]},
                            {'platform_id':'tiktok','metrics':['followers','likes']}
                        ]

    df_attributes = ['ChartMetric Artist ID','Metric Platform','Metric Name','timestp','value','diff']

    f_df = pd.DataFrame(columns=df_attributes)

    cm = ChartMetricAPI()

    artists_df.columns = artists_df.columns.str.replace(' ','_')
    artists_df.columns = artists_df.columns.str.replace('.','_')

    for row in artists_df.itertuples():
        if row.ChartMetric_Artist_ID != np.nan:
            artist_id = row.ChartMetric_Artist_ID
            if from_dt == "":
                from_dt = getattr(row, 'Start_Date', None)
            if to_dt == "":
                to_dt = getattr(row, 'End_Date', None)
            for ref in fan_metric_ref:
                print(artist_id)
                print(ref['platform_id'])
                print(from_dt)
                print(to_dt)
                try:
                    metric_json = cm.getArtistFanMetrics(artist_id, ref['platform_id'], from_dt, to_dt)
                    time.sleep(0.4)
                    if metric_json is not None:
                        if metric_json['obj'] != []:
                            for item in metric_json['obj']:
                                if item in ref['metrics']:
                                    s_df = json_normalize(metric_json['obj'][item])
                                    s_df['ChartMetric Artist ID'] = artist_id
                                    s_df['Metric Platform'] = ref['platform_id']
                                    s_df['Metric Name'] = item
                                    f_df = pd.concat([f_df, s_df])
                except Exception as e:
                    print(f"Error occurred while processing {ref['platform_id']} for artist {artist_id}: {str(e)}")
                    continue

    f_df.rename(columns={"timestp":"Artist Metric Date","value":"Metric Value","diff":"Metric Value Difference"},inplace=True,errors="ignore")
    f_df = f_df.drop_duplicates()
    f_df.drop(columns=['weekly_diff','weekly_diff_percent','monthly_diff','monthly_diff_percent'],inplace=True,errors="ignore")
    f_df['Artist Metric Date'] = f_df['Artist Metric Date'].apply(convertUTCDateFormat)

    return f_df

#Function to extract YT view insights for a collection of ChartMetric artist IDs from the ChartMetric API.
def extractCMArtistYTVMCStats(artists_df):

    yt_views_insights_list = []
    yt_views_mc_list = []
    yt_total_views_list = []

    cm = ChartMetricAPI()

    for index, row in artists_df.iterrows():
      if row['ChartMetric Artist ID'] != np.nan:
        artist_id = int(row['ChartMetric Artist ID'])
        yt_json = cm.getArtistYTViewsandMC(artist_id)
        time.sleep(1)
        for metric_category in yt_json['obj']:
          if metric_category == 'insights':
            for region in yt_json['obj'][metric_category]['cities']:
              for trend in region['trend']:
                metric_dict = {}
                metric_dict['ChartMetric Artist ID'] = artist_id
                metric_dict['Platform'] = 'YouTube'
                metric_dict['Metric Type'] = 'Views Insights'
                metric_dict['Metric Query ID'] = region['params_id']
                metric_dict['Current Date'] = region['timestp']
                metric_dict['Region Name'] = region['name']
                metric_dict['Region Latitude'] = region['lat']
                metric_dict['Region Longitude'] = region['lng']
                metric_dict['Region Country Code'] = region['code2']
                metric_dict['Region City ID'] = region['city_id']
                metric_dict['Region Type'] = region['region']
                metric_dict['Region Current Max Count'] = region['current_max_count']
                metric_dict['Region Current Rank for Artist'] = region['rank'] #The rank of the region for an artist's overall YT views
                metric_dict['Region Historical Views Timestamp'] = trend['timestp']
                metric_dict['Region Historical Views Count'] = trend['monthlyviews']
                yt_views_insights_list.append(metric_dict)

          elif metric_category == 'marketInsights':

                for region in yt_json['obj'][metric_category]['marketCities']:
                    metric_dict = {}

                    metric_dict['ChartMetric Artist ID'] = artist_id
                    metric_dict['Platform'] = 'YouTube'
                    metric_dict['Metric Type'] = 'Views Market Coverage'
                    metric_dict['Metric Query ID'] = region['youtubeforartist_location']
                    metric_dict['Current Date'] = date.today().strftime("%Y-%m-%d")
                    metric_dict['Region Name'] = region['name']
                    metric_dict['Region Latitude'] = region['lat']
                    metric_dict['Region Longitude'] = region['lng']
                    metric_dict['Region Country Code'] = region['code2']
                    metric_dict['Region City ID'] = region['city_id']
                    metric_dict['Region Type'] = region['region']
                    metric_dict['Region Current Number of Views'] = region['value']
                    metric_dict['Region Previous Number of Views'] = region['prev_value']
                    metric_dict['Region Views Market Share'] = region['weight']
                    metric_dict['Region Leading Artist ChartMetric ID'] = region['market_max_cm_artist_id']
                    metric_dict['Region Leading Artist Name'] = region['market_max_artist_name']
                    yt_views_mc_list.append(metric_dict)

          elif metric_category == 'artistTotalViews':

                metric_dict = {}
                metric_dict['ChartMetric Artist ID'] = row['ChartMetric Artist ID']
                metric_dict['Platform'] = 'YouTube'
                metric_dict['Metric Type'] = 'Total Artist Views'
                metric_dict['Current Date'] = date.today().strftime("%Y-%m-%d")
                metric_dict['Current Metric Value'] = yt_json['obj'][metric_category]
                yt_total_views_list.append(metric_dict)

    yt_views_df = pd.DataFrame(yt_views_insights_list)
    yt_mc_df = pd.DataFrame(yt_views_mc_list)
    yt_total_views_df = pd.DataFrame(yt_total_views_list)

    return yt_views_df, yt_mc_df, yt_total_views_df

#Function to extract Social Language Stats for an artist from the ChartMetricAPI.
def extractCMArtistSocialAudienceStats(artists_df,metric_type="stat"):

    if metric_type == "stat":
      social_ref = [
            {'platform':'instagram','stat':'followers','stat_display_name':'Followers'},
            {'platform':'tiktok','stat':'followers','stat_display_name':'Followers'},
            {'platform':'youtube','stat':'followers','stat_display_name':'Followers'}
        ]
    elif metric_type == "language":
      social_ref = [
                {'platform':'instagram','stat':'likes','stat_display_name':'Likes'},
                {'platform':'instagram','stat':'followers','stat_display_name':'Followers'},
                {'platform':'tiktok','stat':'followers','stat_display_name':'Followers'},
                {'platform':'youtube','stat':'followers','stat_display_name':'Followers'},
                {'platform':'youtube','stat':'commenters','stat_display_name':'Commenters'}
      ]
    elif metric_type == "country":
      social_ref = [
            {'platform':'instagram','stat':'likes','stat_display_name':'Likes'},
            {'platform':'instagram','stat':'followers','stat_display_name':'Followers'},
            {'platform':'tiktok','stat':'followers','stat_display_name':'Followers'},
            {'platform':'youtube','stat':'followers','stat_display_name':'Followers'},
            {'platform':'youtube','stat':'commenters','stat_display_name':'Commenters'}
        ]
    elif metric_type == "demographic":
      social_ref = [
            {'platform':'instagram','stat':'likes','stat_display_name':'Likes'},
            {'platform':'instagram','stat':'followers','stat_display_name':'Followers'},
            {'platform':'tiktok','stat':'followers','stat_display_name':'Followers'},
            {'platform':'youtube','stat':'followers','stat_display_name':'Followers'},
            {'platform':'youtube','stat':'commenters','stat_display_name':'Commenters'}
        ]

    f_df = df()

    cm = ChartMetricAPI()

    artists_df.columns = artists_df.columns.str.replace(' ','_')
    artists_df.columns = artists_df.columns.str.replace('.','_')

    for row in artists_df.itertuples():
      if row.ChartMetric_Artist_ID != np.nan:
        artist_id = row.ChartMetric_Artist_ID
        from_dt = getattr(row, 'Start_Date', None)
        to_dt = getattr(row, 'End_Date', None)
        for ref in social_ref:
          stats_json = cm.getArtistSocialAudienceStats(cm_artist_id=artist_id,domain=ref['platform'],audience_type=ref['stat'],stats_type=metric_type,from_date=from_dt,to_date=to_dt)
          time.sleep(0.3)
          if stats_json is not None and stats_json != np.nan:
            if stats_json != []:
              s_df = json_normalize(stats_json['obj'])
              s_df['ChartMetric Artist ID'] = artist_id
              s_df['Platform'] = ref['platform']
              s_df['Metric'] = ref['stat_display_name']
              f_df = pd.concat([f_df,s_df])

    f_df.drop(columns=["avg_likes_per_post","avg_comments_per_post","avg_views_per_post"],inplace=True,errors="ignore")
    return f_df

#Function to extract Monthly Listeners Info from Spotify By City and Country from the ChartMetricAPI.
def extractCMArtistSpotifyListeners(artists_df):

    cm = ChartMetricAPI()
    metrics_list = []

    sp_listeners_metrics = [
        {'dimension':'cities','attributes':[
                {'id':'timestp','column_name':'Metric Date'},
                {'id':'code2','column_name':'Metric Country Code'},
                {'id':'region','column_name':'Metric Type'},
                {'id':'listeners','column_name':'Listeners'},
                {'id':'artist_city_rank','column_name':'Artist City Rank'},
                {'id':'population','column_name':'Region Population'}
            ]
            },
        {'dimension':'countries','attributes':[
                {'id':'timestp','column_name':'Metric Date'},
                {'id':'code2','column_name':'Metric Country Code'},
                {'id':'region','column_name':'Metric Type'},
                {'id':'listeners','column_name':'Listeners'},
                {'id':'artist_city_rank','column_name':'Artist City Rank'},
                {'id':'population','column_name':'Region Population'}]}]

    artists_df.columns = artists_df.columns.str.replace(' ','_')
    artists_df.columns = artists_df.columns.str.replace('.','_')

    for index, row in artists_df.itertuples():
        if row.ChartMetric_Artist_ID != np.nan:
            artist_id = int(row.ChartMetric_Artist_ID)
            from_dt = row.Album_Release_Date
            to_dt = row.End_Date
            sp_json = cm.getArtistSPMonthlyListenersByCity(artist_id,since=from_dt,until=to_dt)
            #time.sleep(0.5)
            for metric in sp_listeners_metrics:
                for region in sp_json['obj'][metric['dimension']]:
                    for record in sp_json['obj'][metric['dimension']][region]:
                        metric_dict = {}
                        metric_dict['ChartMetric Artist ID'] = row.ChartMetric_Artist_ID
                        metric_dict['Platform'] = 'Spotify'
                        metric_dict['Region Name'] = region
                        for attr in metric['attributes']:
                            metric_dict[attr['column_name']] = record[attr['id']]
                        metrics_list.append(metric_dict)

    f_df = pd.DataFrame(metrics_list)
    f_df['Metric Date'] = f_df['Metric Date'].apply(convertUTCDateFormat)

    return f_df

#Function to extract playlist evolution stats by artist, album, or track from the ChartMetricAPI.  NOTE: Currently only geared for artists
def extractCMArtistPlaylistEvolutionStats(artists_df):

    platform_list = ['spotify','youtube']

    f_df = pd.DataFrame()

    artists_df.columns = artists_df.columns.str.replace(' ','_')
    artists_df.columns = artists_df.columns.str.replace('.','_')

    cm = ChartMetricAPI()

    for row in artists_df.itertuples():
        id = row.ChartMetric_Artist_ID
        from_dt = row.Start_Date
        to_dt = row.End_Date
        if id != np.nan:
            stats_json = cm.getPlaylistEvolutionStats('artist',id,since=from_dt,until=to_dt) #Update since Parameter to be oneRPM artist creation date
            #time.sleep(0.2)
            if stats_json is not None:
              if stats_json['obj'] != [] and stats_json['obj'] != {}:
                for platform in platform_list:
                    s_df = json_normalize(stats_json['obj'][platform])
                    s_df.rename(columns={'created_at':'Metric Date','playlist_count':'Artist Playlist Count','ed_playlist_count':'Artist Editorial Playlist Count','playlist_total_reach':'Artist Playlist Total Reach','ed_playlist_total_reach':'Artist Editorial Playlist Total Reach'},inplace=True,errors="ignore")
                    s_df['Platform'] = platform
                    s_df['ChartMetric Artist ID'] = id
                    f_df = pd.concat([f_df,s_df])

    return f_df

#Function to extract ChartMetric playlist stats over time for a given playlist from the ChartMetric API.
def extractCMTrackPlaylistEvolutionStats(track_ids_df):

    f_df = pd.DataFrame()

    platform_list = ['spotify','youtube']

    track_ids_df.columns = track_ids_df.columns.str.replace(' ','_')
    track_ids_df.columns = track_ids_df.columns.str.replace('.','_')

    cm = ChartMetricAPI()

    for row in track_ids_df.itertuples():
        id = row.ChartMetric_Track_ID
        from_dt = row.Start_Date
        to_dt = row.End_Date
        stats_json = cm.getPlaylistEvolutionStats('track',id,since=from_dt,until=to_dt)
        #time.sleep(0.2)
        if stats_json is not None:
            stats_json = stats_json['obj']
            for platform in platform_list:
                if stats_json[platform] != []:
                    s_df = json_normalize(stats_json[platform])
                    s_df['Platform'] = platform
                    s_df['ChartMetric Track ID'] = id

                    f_df = pd.concat([f_df,s_df])

    f_df.rename(columns={'created_at':'Track Metric Date','playlist_count':'Track Playlist Count','ed_playlist_count':'Track Editorial Playlist Count','playlist_total_reach':'Track Playlist Total Reach','ed_playlist_total_reach':'Track Editorial Playlist Total Reach'},inplace=True,errors="ignore")

    return f_df

#Function to extract charts for a given music platform and other subsets (i.e. country, genre, chart type, etc) via the ChartMetric API.  WORK IN PROGRESS.  NOT FINISHED YET
def extractCMCharts(chart_type="tracks",from_dt="",to_dt=""):

    f_df = pd.DataFrame()

    cm=ChartMetricAPI()

    std_countries = [
        'GLOBAL','US','JP','DE','FR','GB','IT','CN','BR','CA', 'ES', 'IN', 'MX', 'NL', 'AU', 'AR', 'KR', 'RU', 'CH', 'TW', 'BE', 'SE', 'AT', 'TR', 'DK', 'HK', 'PL', 'NO', 'SA', 'FI', 'GR', 'ZA', 'TH', 'PT', 'CO', 'IL', 'VE', 'SG', 'ID', 'EG', 'IE', 'CL', 'MY', 'NG', 'PH', 'PE', 'CZ', 'NZ', 'HU', 'UA', 'RO', 'AE', 'KZ', 'VN', 'EC', 'SI', 'GT', 'LB', 'LU', 'DO', 'LK', 'BG', 'SV', 'LT', 'CR', 'CY', 'KE', 'PA', 'BO', 'PY', 'JO', 'LV', 'ZW', 'HN', 'EE', 'KH', 'PG', 'NI', 'MD', 'LA', 'SZ', 'MN']
    std_countries_ex_global = std_countries[1:]
    sc_countries = ['IE', 'AU', 'NL', 'CA', 'GLOBAL', 'US', 'FR', 'NZ', 'GB', 'DE']

    az_genres = [
        "All Genres","Alternative Rock", "Blues", "Broadway & Vocalists", "Children's Music", "Christian", "Classic Rock", "Classical", "Country", "Dance & Electronic", "Folk", "Hard Rock & Metal", "Holiday",
                 "International", "Jazz", "Latin Music", "Miscellaneous", "New Age", "Opera & Vocal", "Pop", "R&B", "Rap & Hip-Hop", "Rock", "Soundtracks"]

    am_genres = [
        "All Genres", "Pop", "Rock", "Dance", "Latino", "K-Pop", "Singer/Songwriter", "Hip-Hop/Rap", "Jazz", "Electronic", "R&B/Soul", "Blues", "Country", "Reggae", "Classical", "Alternative", "World", "Disney",
                 "J-Pop", "Christian & Gospel", "Easy Listening", "Children's Music", "Fitness & Workout", "Soundtrack"]

    bp_genres = [
                  "top-100", "house", "tech-house", "future-house", "electronica-downtempo", "progressive-house", "electro-house", "techno", "glitch-hop", "drum-and-bass", "trance", "reggae-dancehall-dub", "hip-hop-r-and-b",
                 "indie-dance-nu-disco", "hard-dance", "dubstep", "deep-house", "psy-trance", "big-room", "minimal-deep-tech", "funky-groove-jackin-house", "funk-soul-disco", "dance", "breaks", "dj-tools", "hardcore-hard-techno",
                 "leftfield-house-and-techno"
                 ]

    sc_genres = [
                "all-music", "pop", "hiphoprap", "country", "rock", "jazzblues", "classical", "reggae", "house", "folksingersongwriter", "techno", "trance", "disco", "rbsoul", "world", "latin", "piano", "reggaeton",
                 "alternativerock", "indie", "triphop", "electronic", "deephouse", "drumbass", "dancehall", "soundtrack", "ambient", "dubstep", "danceedm", "trap", "metal"]

    if chart_type == "tracks":
      chart_refs = [
        {'platform_id':'airplay','type':'tracks','available_countries':std_countries,'sub_types':[],'durations':['daily','weekly'],'genres':[]},
        {'platform_id':'amazon','type':'tracks','available_countries':std_countries_ex_global,'sub_types':['popular_track','new_track'],'durations':[],'genres':az_genres},
        {'platform_id':'applemusic','type':'tracks','available_countries':std_countries_ex_global,'sub_types':['top','daily'],'durations':[],'genres':am_genres},
        {'platform_id':'beatport','type':'tracks','available_countries':[],'sub_types':[],'durations':[],'genres':bp_genres},
        {'platform_id':'chartmetric','type':'track','available_countries':[],'sub_types':['spotify-top','spotify-viral','applemusic-genre','applemusic-daily','applemusic-albums','itunes','itunes_albums','shazam'],'durations':[],'genres':[]},
        {'platform_id':'deezer','type':'track','available_countries':std_countries_ex_global,'sub_types':[],'durations':[],'genres':[]},
        {'platform_id':'qq','type':'track','available_countries':[],'sub_types':[],'durations':[],'genres':[]},
        {'platform_id':'shazam','type':'track','available_countries':std_countries,'sub_types':[],'durations':[],'genres':[]},
        {'platform_id':'soundcloud','available_countries':sc_countries,'sub_types':['top','trending'],'durations':[],'genres':sc_genres},
        {'platform_id':'spotify','type':'fresh_find','available_countries':[],'sub_types':[],'durations':[],'genres':[]},
        {'platform_id':'spotify','type':'tracks','available_countries':std_countries,'sub_types':['regional','viral'],'durations':['daily','weekly'],'genres':[]},
        {'platform_id':'tiktok','type':'tracks','available_countries':std_countries,'sub_types':[],'durations':[],'genres':[]},
        {'platform_id':'youtube','type':'tracks','available_countries':std_countries,'sub_types':[],'durations':[],'genres':[]},
        {'platform_id':'itunes','type':'tracks','available_countries':std_countries_ex_global,'sub_types':[],'durations':[],'genres':am_genres}
      ]
    elif chart_type == "artists":
      chart_refs = [
          {'platform_id':'twitch','type':'track','available_countries':std_countries,'sub_types':[],'durations':['daily','weekly','monthly'],'genres':[]},
          ]
    elif chart_type == "albums":
      chart_refs = []
    else:
      print("Invalid Chart Type Selected!")
      return None

    start_dt = from_dt
    current_dt = to_dt
    day_cnt = get_day_count(start_dt,current_dt)

    for record_dt in (start_dt + timedelta(n) for n in range(day_cnt)):
      for ref in chart_refs:
        if ref['sub_types'] != [] and ref['available_countries']!= [] and ref['durations']!= [] and ref['genres']!=[]:
          for sub_type in ref['sub_types']:
            for country in ref['available_countries']:
              for duration in ref['durations']:
                for genre in ref['genres']:
                  chart_json = cm.getCharts(chart_platform=ref['platform_id'],chart_type=ref['type'],chart_sub_type=sub_type,country_code=country,countryChart=True,chart_duration=duration,genre=genre)
                  if chart_json is not None:
                    if chart_json['obj']['data'] != []:
                      s_df = json_normalize(chart_json['obj']['data'])
                      f_df = pd.concat([f_df,s_df])
        elif ref['sub_types'] != [] and ref['available_countries']!= [] and ref['durations']!= []:
          for sub_type in ref['sub_types']:
            for country in ref['available_countries']:
              for duration in ref['durations']:
                chart_json = cm.getCharts(chart_platform=ref['platform_id'],chart_type=ref['type'],chart_sub_type=sub_type,country_code=country,countryChart=True,chart_duration=duration)
                if chart_json is not None:
                    if chart_json['obj']['data'] != []:
                      s_df = json_normalize(chart_json['obj']['data'])
                      f_df = pd.concat([f_df,s_df])
        elif ref['sub_types'] != [] and ref['available_countries']!= []:
          for sub_type in ref['sub_types']:
            for country in ref['available_countries']:
              chart_json = cm.getCharts(chart_platform=ref['platform_id'],chart_type=ref['type'],chart_sub_type=sub_type,country_code=country,countryChart=True)
              if chart_json is not None:
                if chart_json['obj']['data'] != []:
                  s_df = json_normalize(chart_json['obj']['data'])
                  f_df = pd.concat([f_df,s_df])
        elif ref['sub_types'] != []:
          for sub_type in ref['sub_types']:
            chart_json = cm.getCharts(chart_platform=ref['platform_id'],chart_type=ref['type'],chart_sub_type=sub_type)
            if chart_json is not None:
              if chart_json['obj']['data'] != []:
                s_df = json_normalize(chart_json['obj']['data'])
                f_df = pd.concat([f_df,s_df])
        else:
          chart_json = cm.getCharts(chart_platform=ref['platform_id'],chart_type=ref['type'])
          if chart_json is not None:
            if chart_json['obj']['data'] != []:
              s_df = json_normalize(chart_json['obj']['data'])
              f_df = pd.concat([f_df,s_df])

    return f_df

#Function to pull Spotify Track Audio Features from Spotify API into a clean dataframe
def extractSPAudioFeatures(sp_track_ids_df):

    total_row_count = sp_track_ids_df.shape[0]

    current_entry = 0
    offset_amount = 99

    all_sp_ids_list = sp_track_ids_df['Spotify ID'].tolist()
    df_attributes = ["id","acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","speechiness","tempo","time_signature","valence"]
    f_df = pd.DataFrame(columns=df_attributes)

    while current_entry <= total_row_count:
        sp_ids_sublist = all_sp_ids_list[current_entry:current_entry+offset_amount]
        try:
          audio_json = sp_client.audio_features(sp_ids_sublist)
          #print(audio_json)
        except sp.SpotifyException:
          print(sp.SpotifyException)
          time.sleep(10)
          audio_json = sp_client.audio_features(sp_ids_sublist)
        finally:
          time.sleep(2)
        s_df = json_normalize(audio_json)
        current_entry += offset_amount
        f_df = pd.concat([f_df,s_df],join="inner")
    return f_df

#Function to pull Spotify Track Audio Analysis Details from the Spotify API into a clean dataframe (WORK IN PROGRESS)
def extractSPAudioAnalyses(sp_track_ids_df):

    final_sp_track_lvl_audio_features = pd.DataFrame(columns=['num_samples','duration','offset_seconds','window_seconds','analysis_sample_rate','analysis_channels','end_of_fade_in','start_of_fade_out','loudness','tempo','tempo_confidence','time_signature','time_signature_confidence','key','key_confidence','mode','mode_confidence','Spotify Track ID'])
    final_sp_audio_sections_df = pd.DataFrame(columns=['start','duration','confidence','loudness','tempo','tempo_confidence','key','key_confidence','mode','mode_confidence','time_signature','time_signature_confidence','Spotify Track ID','Section Number'])
    final_sp_audio_segments_df = pd.DataFrame(columns=['start','duration','confidence','loudness_start','loudness_max_time','loudness_max','loudness_end','pitches','timbre','Spotify Track ID','Segment Number'])
    final_sp_audio_beats_df = pd.DataFrame(columns=['start','duration','confidence','Spotify Track ID','Beat Number'])
    final_sp_audio_bars_df = pd.DataFrame(columns=['start','duration','confidence','Spotify Track ID','Bar Number'])
    final_sp_audio_tatums_df = pd.DataFrame(columns=['start','duration','confidence','Spotify Track ID','Tatum Number'])

    for index, row in sp_track_ids_df.iterrows():
        print(str(index) + ' ' + str(row))

        sp_track_id = row['Spotify ID']

        audio_json = sp_client.audio_analysis(sp_track_id)
        time.sleep(0.5)

        sp_track_lvl_audio_features = json_normalize(audio_json)
        sp_track_lvl_audio_features['Spotify Track ID'] = sp_track_id
        sp_track_lvl_audio_features.drop(['bars','beats','sections','segments','tatums'],inplace=True,errors="ignore")

        final_sp_track_lvl_audio_features = pd.concat([sp_track_lvl_audio_features,final_sp_track_lvl_audio_features],join="outer")

        sp_audio_sections_df = json_normalize(audio_json,record_path=['sections'])
        sp_audio_sections_df['Spotify Track ID'] = sp_track_id
        sp_audio_sections_df['Section Number'] = np.arange(len(sp_audio_sections_df))+1

        final_sp_audio_sections_df = pd.concat([final_sp_audio_sections_df,sp_audio_sections_df],join="outer")

        sp_audio_segments_df = json_normalize(audio_json,record_path=['segments'])
        sp_audio_segments_df['Spotify Track ID'] = sp_track_id
        sp_audio_segments_df['Segment Number'] = np.arange(len(sp_audio_segments_df))+1

        final_sp_audio_segments_df = pd.concat([final_sp_audio_segments_df,sp_audio_segments_df],join="outer")

        sp_audio_beats_df = json_normalize(audio_json, record_path=['beats'])
        sp_audio_beats_df['Spotify Track ID'] = sp_track_id
        sp_audio_beats_df['Beat Number'] = np.arange(len(sp_audio_beats_df))+1

        final_sp_audio_beats_df = pd.concat([final_sp_audio_beats_df,sp_audio_beats_df],join="outer")

        sp_audio_bars_df = json_normalize(audio_json, record_path=['bars'])
        sp_audio_bars_df['Spotify Track ID'] = sp_track_id
        sp_audio_bars_df['Bar Number'] = np.arange(len(sp_audio_bars_df))+1

        final_sp_audio_bars_df = pd.concat([final_sp_audio_bars_df,sp_audio_bars_df],join="outer")

        sp_audio_tatums_df = json_normalize(audio_json, record_path=['bars'])
        sp_audio_tatums_df['Spotify Track ID'] = sp_track_id
        sp_audio_tatums_df['Tatum Number'] = np.arange(len(sp_audio_tatums_df))+1

        final_sp_audio_tatums_df = pd.concat([final_sp_audio_tatums_df,sp_audio_tatums_df],join="outer")

    return final_sp_track_lvl_audio_features, final_sp_audio_sections_df, final_sp_audio_segments_df, final_sp_audio_beats_df, final_sp_audio_bars_df, final_sp_audio_tatums_df

#Function to pull Spotify Track Features from the Spotify API into clean dataframe
def extractSPTrackMetadata(sp_track_ids_df):
    total_row_count = sp_track_ids_df.shape[0]

    current_entry = 0
    offset_amount = 49

    all_sp_ids_list = sp_track_ids_df['Spotify ID'].tolist()

    track_attributes = ['id',
                        'popularity',
                        'available_markets',
                        'duration_ms',
                        'explicit',
                        'is_local'#,
                        #'track_number',
                        #'disc_number',
                        #'album.album_type',
                        #'album.id',
                        #'album.release_date'
    ]

    f_df = pd.DataFrame(columns=track_attributes)

    while current_entry <= total_row_count:
        print(all_sp_ids_list[current_entry:current_entry+offset_amount])
        track_json = sp_client.tracks(all_sp_ids_list[current_entry:current_entry+offset_amount])
        s_df = json_normalize(track_json['tracks'])
        f_df = pd.concat([f_df,s_df],join="inner")
        current_entry += offset_amount
    return f_df

#Function to generate the dates for merging timeseries ChartMetric data into oneRPM data.
def generateRecordDates(combined_data_df):

  combined_data_df[['ChartMetric Track ID','Album Release Date','End Date']]

  track_record_date_df = pd.DataFrame()

  combined_data_df.columns = combined_data_df.columns.str.replace(' ','_')
  combined_data_df.columns = combined_data_df.columns.str.replace('.','_')

  for row in combined_data_df.itertuples():

    record_date_df = pd.DataFrame(pd.date_range(start=row.Album_Release_Date,end=row.End_Date),columns=["Record Date"])
    record_date_df['ChartMetric Track ID'] = row.ChartMetric_Track_ID

    track_record_date_df = pd.concat([track_record_date_df,record_date_df])

  track_record_date_df['Record Date'] = track_record_date_df['Record Date'].astype(str)
  return track_record_date_df

#Function to extract all the playlists that an artist's tracks appear on.  WORK IN PROGRESS.  NOT FINISHED YET
def extractCMArtistPlaylists(artists_df):
    cm = ChartMetricAPI()
    playlist_list = []
    for row in artists_df.itertuples():
        artist_id = cm.getCMArtistIdByName(row.Artist)
        playlists = cm.getArtistPlaylists(artist_id)
        for playlist in playlists:
            playlist_dict = {'artist': row.Artist}
            for track in playlist['tracks']:
                playlist_dict.update({
                    'Playlist ID': playlist['id'],
                    'Playlist Name': playlist['name'],
                    'Track Peak Position': track['peak_position'],
                    'Current Track Position': track['position'],
                    'Monthly Change in Followers': playlist['fdiff_month'],
                    'Weekly Change in Followers': playlist['fdiff_week'],
                    'Playlist Track Addition Date': track['added_at'],
                    'Playlist Track Removal Date': track['removed_at'],
                    'Playlist Track Inclusion Period': track['period'],
                    'Playlist Latest Followers Count': playlist['followers'],
                    'Playlist Country Code': playlist['code2'],
                    'Playlist Curator': playlist['curator_name'],
                    'Personalized Playlist': playlist['personalized'],
                    'Official Playlist': playlist['official'],
                    'Playlist Tags': ', '.join(playlist['tags']),
                    'Playlist Latest Track Count': playlist['num_tracks'],
                    'Playlist Active Ratio': playlist['active_ratio'],
                    'Last Track Position on Playlist': playlist['last_position'],
                    'Peak Track Position on Playlist': playlist['peak_position'],
                    'Track ISRC': track['isrc'],
                    'Platform Track ID': track['spotify_track_ids'],
                    'ChartMetric Track ID': track['cm_track'],
                    'ChartMetric Artist ID': track['cm_artist'],
                    'Platform Track Genres': ', '.join(track['genres'])
                })
                playlist_list.append(playlist_dict.copy())
    return pd.DataFrame(playlist_list)

#Function to pull airplay radio stats for an artist, track, or album via the ChartMetric API
def extractCMAirplayTimeSeriesStats(target_df,id_type):
    f_df = pd.DataFrame()
    cm = ChartMetricAPI()

    target_df.columns = target_df.columns.str.replace(' ','_')
    target_df.columns = target_df.columns.str.replace('.','_')

    for row in target_df.itertuples():
      if id_type == 'artist':
        id_value = row.ChartMetric_Artist_ID
      elif id_type == 'track':
        id_value = row.ChartMetric_Track_ID
      else:
        print("Invalid ID Type selected.  Please select one of the following valid options: 'artist', 'track'")
        return None
      since_dt = row.Start_Date
      airplay_json = cm.getAirPlayTimeSeriesStats(id_type,id_value,since_dt)
      time.sleep(0.3)
      if airplay_json is not None and airplay_json != {}:
        if airplay_json['obj'] != []:
          s_df = json_normalize(airplay_json['obj'])
          if id_type == 'track':
            s_df['ChartMetric Track ID'] = id_value
          else:
            s_df['ChartMetric Artist ID'] = id_value
          f_df = pd.concat([f_df,s_df])

    f_df['Metric Date'] = f_df['air_date'].astype('int').astype("datetime64[ms]").dt.strftime("%Y-%m-%d")
    f_df.drop(columns=['air_date'],inplace=True,errors="ignore")
    if id_type == 'track':
      f_df.rename(columns={'spins':'cm.track.numberAirplaySpins'},inplace=True,errors="ignore")
    else:
      f_df.rename(columns={'spins':'cm.artist.numberAirplaySpins'},inplace=True,errors="ignore")
    return f_df

#Function to run an extraction process across a large dataframe with chunks of extracted data saved periodically.  WORK IN PROGRESS. NOT FINISHED YET.
def runBulkExtractionProcess(input_df,extract_func):
  ''''''

# ARTIST SEARCH

In [ ]:
#@title Add an artist or a list of artists to search in Chartmetric.

# List of artists to query
#artist_list = ["Oblivion's Mighty Trash", "Granuja", "Penyair", "Crudo Means Raw"]
artist_list = ["Yo No La Tengo"]


In [ ]:
#@title Add an artist list from a csv.
# Read the CSV file with artist names

# artist_df = pd.read_csv('/content/drive/MyDrive/-DataPlayground/data_inputs/perf_artists.csv', header=0) # limit rows with nrows = , or skiprows=range(1, 853),
# Extract the artist names from the "artist" column
# artist_names = artist_df['artist'].tolist()

In [ ]:
#@title Get Tracks and their Metadata from an Artist or list of Artists and Match against ONErpm PERF DB
# GET ALL THE TRACKS AND ITS METADATA FROM AN ARTIST OR A LIST OF ARTISTS
# Initialize ChartMetric API client
cm = ChartMetricAPI()

# Path to the file containing the list of artists
#file_path = "/content/drive/MyDrive/-DataPlayground/artists.csv"

# Read the file into a dataframe
#artists_df = pd.read_csv(file_path, skiprows=0, nrows=150)

# Extract the list of artists as a Python list
#artist_list = artists_df["artist"].tolist()

# Initialize an empty list to store the results
artist_track_list = []

# Loop through each artist and pull their tracks
for artist in artist_list:
    # Search for the artist in ChartMetric
    artist_id = cm.getCMArtistIdByName(artist)
    # Use the ChartMetric ID to pull the artist's tracks
    try:
        artist_tracks = cm.getArtistTracks(artist_id)
        # Check if artist_tracks is not None
        if artist_tracks is None:
            print(f"No tracks found for {artist}")
            continue
    except Exception as e:
        print(f"Could not retrieve tracks for {artist}: {e}")
        continue
    # Flatten the 'obj' column
    artist_tracks_obj = artist_tracks['obj']
    flattened_tracks_obj = pd.json_normalize(artist_tracks_obj)
    # Add the artist name as a column
    flattened_tracks_obj['artist'] = artist
    # Append the flattened tracks to the list
    artist_track_list.append(flattened_tracks_obj)

# Concatenate all of the results into a single dataframe
all_tracks_df = pd.concat(artist_track_list, ignore_index=True)

# Print the resulting dataframe
# print(all_tracks_df.head())

# Crear conexión con Redshift

creds = json.loads(requests.get("https://drive.google.com/uc?export=download&id=1upW1Hyque4cXgmp8bP-_vmBCcen5ZH5e").content)
key = "SWEixBZOLYArJ-XPSBuK0b-UOrlENANMMmjcuRpyFP4="

# value of key is assigned to a variable
f = Fernet(key)
decoded_creds = {}

for key in creds.keys():
  decoded_key = f.decrypt(key.encode()).decode()
  decoded_value = f.decrypt(creds[key].encode()).decode()
  decoded_creds[decoded_key] = decoded_value

conn = psycopg2.connect(
    host=decoded_creds["host"],
    port=decoded_creds["port"],
    database=decoded_creds["database"],
    user=decoded_creds["user"],
    password=decoded_creds["password"]
)


# Leer el dataframe original con los ISRC

df_original = all_tracks_df

# Convertir columna ISRC a tipo string para poder concatenar más adelante.

df_original['isrc'] = df_original['isrc'].astype(str)

# Extraer solo los ISRC

isrc_serie = df_original['isrc']
isrc_serie = np.unique(isrc_serie)

# crear un string con los ISRC para incluir en la consulta SQL

isrc_string = ",".join(["'" + i + "'" for i in isrc_serie])

# crear la consulta SQL
query = f"SELECT comp_id, isrc FROM comp.perf WHERE isrc IN ({isrc_string})"

# ejecutar la consulta y leer los resultados en un dataframe
df_result = pd.read_sql(query, conn)

# Renombra la columna ISRC para que haga match con el query de redshift
df_original.rename(columns={'ISRC': 'isrc'}, inplace=True)

# Haz la unión entre df_original y df_redshift en base a la columna ISRC
df_merged = df_original.merge(df_result, on='isrc', how='left')

# Agrega una columna 'Match' al dataframe resultante
df_merged['Match'] = np.where(df_merged['comp_id'].notnull(), 'MATCH', 'NO MATCH')

# Escribir el archivo resultante
current_date = datetime.now().strftime("%Y-%m-%d")
filename = f"/content/drive/Shared drives/Global | Rights Management/- Ops IT/ISRC_Finder/{artist_list[0].replace(' ', '_')}_{current_date}_matched.csv"
df_merged.to_csv(filename, index=False)

# Muestra solo los registros con ISRC que tienen match
# print(df_merged[df_merged['Match'] == 'MATCH'])

# Muestra solo los registros con ISRC que NO tienen match
# print(df_merged[df_merged['Match'] == 'NO MATCH'])

conn.close()

Files will be in this folder:
[ISRC Finder](https://drive.google.com/drive/folders/10u_f2NRWhwRgdKBPGOTya7aw5JxiiYId)

In [ ]:
#@title Get Tracks and their Metadata from an Artist or list of Artists with NO Match against ONErpm PERF DB

import pandas as pd
import os
from datetime import date

# Assuming ChartMetricAPI class and necessary functions are already defined
cm = ChartMetricAPI()

def get_tracks_and_metadata_for_artists(artist_list, base_directory):
    for artist_name in artist_list:
        # Creating the directory path for the artist
        artist_directory = os.path.join(base_directory, artist_name, "track_stats")
        os.makedirs(artist_directory, exist_ok=True)

        # File path for the CSV
        output_csv_file = os.path.join(artist_directory, f"{artist_name}_tracks.csv")

        # Initialize DataFrame for storing tracks
        artist_tracks_df = pd.DataFrame()

        # Search for the artist in ChartMetric and get the artist ID
        artist_id = cm.getCMArtistIdByName(artist_name)
        if artist_id is None:
            print(f"Artist {artist_name} not found in ChartMetric.")
            continue

        # Use the ChartMetric ID to pull the artist's tracks
        try:
            artist_tracks_response = cm.getArtistTracks(artist_id, limit=100)
            if artist_tracks_response is None:
                print(f"No tracks found for artist {artist_name}.")
                continue
        except Exception as e:
            print(f"Error retrieving tracks for artist {artist_name}: {e}")
            continue

        # Process the response and convert to DataFrame
        artist_tracks_obj = artist_tracks_response['obj']
        tracks_df = pd.json_normalize(artist_tracks_obj)
        tracks_df['artist'] = artist_name  # Add artist name to the DataFrame

        # Append to the artist's DataFrame
        artist_tracks_df = pd.concat([artist_tracks_df, tracks_df], ignore_index=True)

        # Save the DataFrame to CSV
        artist_tracks_df.to_csv(output_csv_file, index=False)
        print(f"Data for {artist_name} saved to {output_csv_file}")


base_directory = "/content/drive/Shared drives/Global | Rights Management/- BizDev A&R/Sourcing Leads"
get_tracks_and_metadata_for_artists(artist_list, base_directory)


Access Token Request Status: 200
Data for Yo No La Tengo saved to /content/drive/Shared drives/Global | Rights Management/- BizDev A&R/Sourcing Leads/Yo No La Tengo/track_stats/Yo No La Tengo_tracks.csv


Files will be in this folder:
[Sourcing Leads](https://drive.google.com/drive/folders/1bboGm52subMr6hmQtdpfTDS5p3Ce80lb)

In [ ]:
#@title Get Track DSP Stats (last three Yrs) from an Artist or Artist List

import pandas as pd
import os
from datetime import date

# Assuming ChartMetricAPI class and necessary functions are already defined
cm = ChartMetricAPI()

def get_track_stats_over_time_with_transformation(artist_list, platforms, start_date, end_date, base_output_folder):
    final_df = pd.DataFrame()

    for artist in artist_list:
        artist_id = cm.getCMArtistIdByName(artist)
        if not artist_id:
            print(f"Artist {artist} not found.")
            continue

        artist_tracks = cm.getArtistTracks(artist_id)
        if not artist_tracks:
            print(f"No tracks found for artist {artist}")
            continue

        tracks_df = pd.json_normalize(artist_tracks['obj'])

        for _, track in tracks_df.iterrows():
            cm_track_id = track['id']
            track_name = track['name']
            isrc = track['isrc']

            for platform in platforms:
                track_stats = cm.getTrackStats(cm_track_id, platform, since=start_date, until=end_date)
                for item in track_stats['obj']:
                  stats_data = []
                  for stat in item['data']:
                      # Ensure 'value' key exists in stat
                      if 'value' in stat:
                          stats_data.append({
                              'Artist': artist,
                              'Track': track_name,
                              'ISRC': isrc,
                              'Platform': platform,
                              'Value': stat['value'],
                              'Date': pd.to_datetime(stat['timestp']).strftime('%Y-%m-%d')
                          })
                  if stats_data:  # Check if stats_data is not empty
                      temp_df = pd.DataFrame(stats_data)
                      temp_df['Value_Diff'] = temp_df['Value'].diff()  # Calculate the difference
                      final_df = pd.concat([final_df, temp_df], ignore_index=True)
                  else:
                      print(f"No valid data for track {track_name} on {platform}")

        # Create artist specific folder and track_stats subfolder
        artist_folder = os.path.join(base_output_folder, artist.replace(" ", "_"))
        track_stats_folder = os.path.join(artist_folder, "track_stats")
        os.makedirs(track_stats_folder, exist_ok=True)

        # Save data to a single CSV file within artist's folder
        filename = f"{artist.replace(' ', '_')}_track_stats_{start_date}_{end_date}.csv"
        filepath = os.path.join(track_stats_folder, filename)
        final_df.to_csv(filepath, index=False)
        print(f"Data saved to {filepath}")

# Example Usage
platforms = ['spotify', 'youtube', 'soundcloud','shazam','tiktok','genius']
end_date = date.today()
start_date = end_date - timedelta(days=3*365)
base_output_folder = "/content/drive/Shared drives/Global | Rights Management/- BizDev A&R/Sourcing Leads"
get_track_stats_over_time_with_transformation(artist_list, platforms, start_date, end_date, base_output_folder)



Access Token Request Status: 200
No valid data for track Todo Mal on soundcloud
No valid data for track Todo Mal on genius
No valid data for track En la Cara on youtube
No valid data for track En la Cara on soundcloud
No valid data for track En la Cara on genius
No valid data for track En el Mar No Eres Nadie on youtube
No valid data for track En el Mar No Eres Nadie on soundcloud
No valid data for track En el Mar No Eres Nadie on genius
No valid data for track Incurable on youtube
No valid data for track Incurable on soundcloud
No valid data for track Incurable on genius
No valid data for track Faruk Urrutia on youtube
No valid data for track Faruk Urrutia on soundcloud
No valid data for track Faruk Urrutia on genius
No valid data for track La Alegría de Permanecer Enterrado on youtube
No valid data for track La Alegría de Permanecer Enterrado on soundcloud
No valid data for track La Alegría de Permanecer Enterrado on genius
No valid data for track Amos y Señores on soundcloud
No vali

Files will be in this folder:
[Sourcing Leads](https://drive.google.com/drive/folders/1bboGm52subMr6hmQtdpfTDS5p3Ce80lb)

# ISRC STATS SEARCH

In [ ]:
#@title ISRC List
# List of artists to query
isrc_list =['QZK6Q2096741',
'QZK6G2043336',
'QZMEN2054311',
'QZNWV2059037',
'QZNWZ2025819',
'QZFZ42196829',
'QZFZ42146095',
'QZDA52117987',
'QZES62185297',
'QZES92178778',
'QZK6Q2056698',
'QZK6L2101986',
'QMFME2188461',
'QMDA62109694',
'QZNWS2131028',
'QZNWT2125277',
'QZRP42132193',
'QZDA72269490'
]


In [ ]:
#@title Retrieve Track DSP Stats from a list of ISRC's

import pandas as pd
import os
from datetime import date, timedelta

# Assuming ChartMetricAPI class and necessary functions are already defined
cm = ChartMetricAPI()

def get_track_stats_by_isrc(isrc_list, platforms, base_output_folder):
    final_df = pd.DataFrame()

    # Dynamic date range setup
    end_date = date.today()
    start_date = end_date - timedelta(days=3*365)  # 3 years before today

    for isrc in isrc_list:
        track_ids_response = cm.getTrackIds(isrc, 'isrc')
        if not track_ids_response or 'obj' not in track_ids_response or not track_ids_response['obj']:
            print(f"No track found for ISRC {isrc}")
            continue

        cm_track_id = track_ids_response['obj'][0]['chartmetric_ids'][0]

        # Fetch track metadata using Chartmetric ID
        track_metadata = cm.getTrackMetadata(cm_track_id)
        track_name = track_name = track_metadata['obj']['name']  # Get track name
        artist = track_metadata['obj']['artists'][0]['name']  # Get artist name from the first artist in the array

        for platform in platforms:
            track_stats = cm.getTrackStats(cm_track_id, platform, since=start_date.strftime("%Y-%m-%d"), until=end_date.strftime("%Y-%m-%d"))
            if track_stats and 'obj' in track_stats:
                # Transforming the 'data' column
                for item in track_stats['obj']:
                    stats_data = []
                    for stat in item['data']:
                        if 'value' in stat:
                            stats_data.append({
                                'Artist': artist,
                                'Track': track_name,
                                'ISRC': isrc,
                                'Platform': platform,
                                'Value': stat['value'],
                                'Date': pd.to_datetime(stat['timestp']).strftime('%Y-%m-%d')
                            })
                    if stats_data:
                        temp_df = pd.DataFrame(stats_data)
                        temp_df['Value_Diff'] = temp_df['Value'].diff()  # Calculate the difference
                        final_df = pd.concat([final_df, temp_df], ignore_index=True)

    today_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_directory = os.path.join(base_output_folder, "ISRC_Search", f"track_search_{today_datetime}")
    os.makedirs(output_directory, exist_ok=True)

    # Save the final DataFrame to a single CSV file
    filename = f"all_tracks_stats_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}.csv"
    filepath = os.path.join(output_directory, filename)
    final_df.to_csv(filepath, index=False)
    print(f"Data saved to {filepath}")

platforms = ['spotify', 'youtube', 'soundcloud','shazam','tiktok','genius']
base_output_folder = "/content/drive/Shared drives/Global | Rights Management/- BizDev A&R/Sourcing Leads"
get_track_stats_by_isrc(isrc_list, platforms, base_output_folder)


Access Token Request Status: 200


KeyboardInterrupt: 

Files will be saved here: [ISRC_Stats](https://drive.google.com/drive/folders/1A4oAMqBIQOMe4nyqKsomXDpPgcHyf-Q7)

Unitary values are defined [here](https://drive.google.com/drive/folders/1oS_OEYS5nzutDVIbjOgqHHZ38FGt0E-U)
